
## Dog Puppulation in Zürich: A Geospatial Neighborhood Analysis

### Introduction

#### Problem Statement:
Can we develop a *pawsome*, data-driven model by the end of January 2024 that predicts what the dog *puppulation* density will be this year across Zürich’s 34 neighborhoods, with a Mean Absolute Error of less than 10%, using time series cross validation, to provide valuable insights for urban planning, pet-related businesses, and community welfare?


#### Context:
Following the City Council Resolution to override the Law on the Keeping of Dogs, the City of Zürich has embarked on a comprehensive exploration of dog *puppulation* dynamics in its neighborhoods. This initiative, prompted by that regulatory shift, aims to sniff out patterns in dog *puppulation* density that impact urban planning, business opportunities, and the overall welfare of our furry companions and their owners. The study leverages data from **2015** to **2020** to improve urban planning, boost pet-related business ventures, and foster community welfare through a better understanding of dog *puppulation* density patterns. This study is vital in this new era for Zürich, providing practical recommendations for the near future. The aim is to develop a data-driven model that reliably predicts the dog *puppulation* density across Zürich’s 34 neighborhoods in the near future.


#### Criteria for Success:
Our goal is to *dig up* clear patterns of dog *puppulation* density in Zürich’s neighborhoods, laying the groundwork for informed future predictions. We aim to *unleash* the potential of our predictive models, forecasting 2024 dog *puppulation* density patterns in Zürich with a Mean Absolute Error of less than 10%. Achieving this would be a *pawsitive* step towards informed future urban strategies.


#### Constraints within Solution Space:
- **Temporal Scope**: The study is confined to the years with full data availability across all datasets (2015-2020)
- **Spatial Resolution**: The study focuses on dog *puppulation* density at the neighborhood level. This may not capture variations within neighborhoods or between smaller areas.
- **Generalizability**: The findings of this study are specific to Zürich and may not be applicable to other cities or regions with different demographic, economic, and cultural contexts.


#### Stakeholders:
- **City Planners and Local Authorities:** Empower data-driven decision-making to enhance urban living conditions.
- **Business Enterprises:** Guide service offerings and marketing strategies.
- **Dog Owners:** Offer insights into community resources and pet care options.


#### Key Data Sources:
- **Geospatial Boundaries:** [Zürich Statistical Quarters](https://data.stadt-zuerich.ch/dataset/geo_statistische_quartiere)
- **Dog Ownership Records:** [Dog Owners Dataset](https://data.stadt-zuerich.ch/dataset/sid_stapo_hundebestand_od1001/download/KUL100OD1001.csv)
- **Demographic Statistics:** [Population Dataset](https://data.stadt-zuerich.ch/dataset/bev_bestand_jahr_quartier_alter_herkunft_geschlecht_od3903/download/BEV390OD3903.csv)
- **Economic Indicators:** [Income Dataset](https://data.stadt-zuerich.ch/dataset/fd_median_einkommen_quartier_od1003/download/WIR100OD1003.csv)
- **Household Dynamics:** [Household Size Dataset](https://data.stadt-zuerich.ch/dataset/bev_hh_haushaltsgroesse_quartier_seit2013_od3806/download/BEV380OD3806.csv)

#### Analytical Objectives:
- **Understand the Relationship**: Dig into the relationship between demographic factors and dog *puppulation* density across Zürich’s neighborhoods.
- **Identify Trends and Clusters**: Track and map out the spatial and temporal trends of dog *puppulation* density. Identify spatial clusters of high and low dog *puppulation* density.
- **Predict Future Trends**: Predict the near-future trends of dog *puppulation* density using historical data, aiming for a Mean Absolute Error of less than 10%. This includes forecasting where Zürich’s dog *puppulation* will be booming across its 34 neighborhoods in the immediate future.


### Imports & Configurations

This section includes the necessary imports for libraries, configuration settings for dataframes and visualizations. These components establish the foundational setup for subsequent data analysis and exploration. 


In [1]:
# Standard libraries
from functools import partial

from IPython.display import clear_output

import math

from PIL import ImageDraw, Image  # For image processing

from urllib.request import urlopen


# Related third party imports

from bokeh.models import FixedTicker, NumeralTickFormatter

import cartopy.crs as ccrs  # For cartographic projections and geographic plots

import colorcet as cc  # Additional color palettes

from esda.moran import Moran, Moran_Local  # Spatial autocorrelation statistics

from fiona.io import ZipMemoryFile

import geopandas as gpd

import geoviews as gv

import holoviews as hv

from holoviews import streams

import hvplot.pandas  # noqa

from matplotlib import pyplot as plt

import libpysal as lps  # Spatial analysis library

import numpy as np

import pandas as pd

import panel as pn

import panel.widgets as pnw
from pmdarima import auto_arima  # For determining ARIMA orders

import seaborn as sns

from splot.esda import plot_local_autocorrelation
from sklearn import metrics  # For evaluating model performance
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import umap


from thefuzz import fuzz  # For string matching
from tqdm.notebook import tqdm  # Progress bars

from wordcloud import WordCloud  # For generating word cloud visualizations


# Local application/library specific imports

import helper_functions as hf  # Custom helper functions for this project

from translate_app import translate_list_to_dict


clear_output()

In [2]:
# Additional configurations for visualization libraries
gv.extension("bokeh")
hv.extension("bokeh")
hvplot.extension("bokeh")
pn.extension(template="fast", nthreads=4, sizing_mode="stretch_width")
clear_output()

In [3]:
# Pandas display options
# Disable warnings for chained assignments
pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 50
pd.options.display.max_rows = 100

# Seaborn style setting
sns.set_style("whitegrid")

# Panel configuration for improved interactivity performance
pn.config.throttled = True

# Clear any output created by the extensions and settings
clear_output()

### Data Description

This project utilizes various datasets to reveal the relationship between dog owner geodemographic factors and dog population density in Zurich. 




<table>
    <thead>
        <tr>
            <th>Dataset</th>
            <th>Source URL</th>
            <th>Original Source</th>
            <th>Description</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td><a href="#Zurich-Statistical-Districts-Geospatial-Data">Zurich Districts Data</a></td>
            <td><a href="https://data.stadt-zuerich.ch/dataset/geo_statistische_quartiere">Link</a></td>
            <td><a href="https://data.stadt-zuerich.ch/dataset/geo_statistische_quartiere">Stadt Zürich</a></td>
            <td>Statistical Quarters</td>
        </tr>
        <tr>
            <td><a href="#Zurich-Dogs-Dataset">Zurich Dogs Data</a></td>
            <td><a href="https://data.stadt-zuerich.ch/dataset/sid_stapo_hundebestand_od1001/download/KUL100OD1001.csv">Link</a></td>
            <td><a href="https://data.stadt-zuerich.ch/dataset/sid_stapo_hundebestand_od1001">Stadt Zürich</a></td>
            <td>Dog populations of the City of Zurich since 2015.</td>
        </tr>
        <tr>
            <td><a href="#Zurich-Population-Dataset">Zurich Population Data</a></td>
            <td><a href="https://data.stadt-zuerich.ch/dataset/bev_bestand_jahr_quartier_alter_herkunft_geschlecht_od3903/download/BEV390OD3903.csv">Link</a></td>
            <td><a href="https://data.stadt-zuerich.ch/dataset/bev_bestand_jahr_quartier_alter_herkunft_geschlecht_od3903">Stadt Zürich</a></td>
            <td>Population by neighbourhood, origin, sex and age, since 1993.</td>
        </tr>
        <tr>
            <td><a href="#Zurich-Income-Data">Zurich Income Data</a></td>
            <td><a href="https://data.stadt-zuerich.ch/dataset/fd_median_einkommen_quartier_od1003/download/WIR100OD1003.csv">Link</a></td>
            <td><a href="https://data.stadt-zuerich.ch/dataset/fd_median_einkommen_quartier_od1003">Stadt Zürich</a></td>
            <td>Median income of taxable individuals by year, tax rate and urban district, since 1999</td>
        </tr>
        <tr>
            <td><a href="#Zurich-Household-Dataset">Zurich Household Data</a></td>
            <td><a href="https://data.stadt-zuerich.ch/dataset/bev_hh_haushaltsgroesse_quartier_seit2013_od3806/download/BEV380OD3806.csv">Link</a></td>
            <td><a href="https://data.stadt-zuerich.ch/dataset/bev_hh_haushaltsgroesse_quartier_seit2013_od3806">Stadt Zürich</a></td>
            <td>Private households by household size and urban district, since 2013.</td>
        </tr>
    </tbody>
</table>

<p>These datasets collectively enable a comprehensive analysis of dog ownership trends in Zurich.</p>


### Data Loading
First, we load in all of the datasets. 

To enhance readability and ensure consistency across datasets, original column names were translated from German to English and standardized to snake case using our `sanitize_df_column_names` helper function. This transformation facilitates a cleaner, more uniform `pd.DataFrame` structure for analysis.

We then inspect the columns and select the ones we would like to keep for our analysis. We also rename the columns to make them more readable and consistent across datasets. 



#### Zurich Statistical Districts Geospatial Data

This first geodataset comes as a compressed file containing 3 geojson files.

1. `z_gdf_0`: point geometry data at the ideal position for placing a number label on the polygon map.

2. `z_gdf_1`: polygon geometry data specifically for visual representation in cartography i.e.maps.

3. `z_gdf_2`: polygon geometry data recommended for use for accurate geometry calculations, like spatial joins or area calculations.

Together these three files provide excellent geodedic information on the geographical region of Zürich for our analysis.

In [4]:
# Define the URL for the Zurich Statistical Quarters geospatial data ZIP file.
zip_gdf_url = "https://storage.googleapis.com/mrprime_dataset/zurich/zurich_statistical_quarters.zip"

# Load the geospatial data into Zurich Geo DataFrames.Would you prefer if we do
zurich_geo_dicts = hf.get_gdf_from_zip_url(zip_gdf_url)

# Rename keys in the Zurich Geo DataFrames with a prefix.
z_gdf = hf.rename_keys(zurich_geo_dicts, prefix="z_gdf_")

# Display the information and a sample of data from each GeoDataFrame in the z_gdf dictionary
for key in z_gdf.keys():
    print(f"\nInformation for {key}:")
    z_gdf[key].info()
    print(f"Sample data from {key}:")
    display(z_gdf[key].sample(3))


Information for z_gdf_0:
<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 34 entries, 0 to 33
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   geometry  34 non-null     geometry
 1   objid     34 non-null     object  
 2   name      34 non-null     object  
 3   kuerzel   34 non-null     object  
 4   ori       34 non-null     int64   
 5   hali      34 non-null     object  
 6   vali      34 non-null     object  
dtypes: geometry(1), int64(1), object(5)
memory usage: 2.0+ KB
Sample data from z_gdf_0:


,geometry,objid,name,kuerzel,ori,hali,vali
28,POINT (8.53208 47.33992),29,Wollishofen,21,0,1,2
4,POINT (8.52337 47.39722),5,Wipkingen,102,0,1,2
27,POINT (8.51073 47.33148),28,Leimbach,23,0,1,2



Information for z_gdf_1:
<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 34 entries, 0 to 33
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   geometry  34 non-null     geometry
 1   objectid  34 non-null     int64   
 2   objid     34 non-null     object  
 3   qnr       34 non-null     int64   
 4   qname     34 non-null     object  
 5   knr       34 non-null     int64   
 6   kname     34 non-null     object  
dtypes: geometry(1), int64(3), object(3)
memory usage: 2.0+ KB
Sample data from z_gdf_1:


,geometry,objectid,objid,qnr,qname,knr,kname
24,"POLYGON ((8.58215 47.38788, 8.58219 47.38794, ...",24,11,71,Fluntern,7,Kreis 7
3,"POLYGON ((8.54795 47.36500, 8.54781 47.36501, ...",4,31,81,Seefeld,8,Kreis 8
21,"POLYGON ((8.52717 47.40668, 8.52688 47.40709, ...",17,21,102,Wipkingen,10,Kreis 10



Information for z_gdf_2:
<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 34 entries, 0 to 33
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   geometry  34 non-null     geometry
 1   objid     34 non-null     object  
 2   qnr       34 non-null     int64   
 3   qname     34 non-null     object  
 4   knr       34 non-null     int64   
 5   kname     34 non-null     object  
dtypes: geometry(1), int64(2), object(3)
memory usage: 1.7+ KB
Sample data from z_gdf_2:


,geometry,objid,qnr,qname,knr,kname
11,"POLYGON ((8.55353 47.39929, 8.55365 47.39917, ...",2,122,Schwamendingen-Mitte,12,Kreis 12
33,"POLYGON ((8.49762 47.38834, 8.49830 47.38814, ...",9,44,Hard,4,Kreis 4
24,"POLYGON ((8.53810 47.37375, 8.53810 47.37363, ...",31,13,Lindenhof,1,Kreis 1


#### Zurich Dogs Dataset

In [5]:
zurich_dog_data_link = "https://data.stadt-zuerich.ch/dataset/sid_stapo_hundebestand_od1001/download/KUL100OD1001.csv"
zurich_dog_data_link = (
    "https://storage.googleapis.com/mrprime_dataset/zurich/zurich_dogs.csv"
)
# InfoDataFrame is a custom class that inherits from pandas.DataFrame and our InfoMixin
zurich_dog_data = hf.InfoDataFrame(pd.read_csv(zurich_dog_data_link))
zurich_dog_data.limit_info()

zurich_dog_data = hf.sanitize_df_column_names(zurich_dog_data)
zurich_dog_data.limit_info()
zurich_dog_data.sample(3)


Total number of columns: 32
<class 'helper_functions.InfoDataFrame'>
RangeIndex: 70967 entries, 0 to 70966
Columns: 32 entries, StichtagDatJahr to AnzHunde
dtypes: int64(19), object(13)
memory usage: 17.3+ MB

Only showing info for 8 columns, chosen at random.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70967 entries, 0 to 70966
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   KreisSort           70967 non-null  int64 
 1   AlterVHundSort      70967 non-null  int64 
 2   AlterV10Cd          70967 non-null  int64 
 3   RasseMischlingSort  70967 non-null  int64 
 4   HalterId            70967 non-null  int64 
 5   AlterVHundCd        70967 non-null  int64 
 6   QuarSort            70967 non-null  int64 
 7   SexHundLang         70967 non-null  object
dtypes: int64(7), object(1)
memory usage: 4.3+ MB

Total number of columns: 32
<class 'helper_functions.InfoDataFrame'>
RangeIndex: 70967 entries, 0 t

,deadline_date_year,data_status_cd,holder_id,age_v_10_cd,age_v_10_long,age_v_10_sort,sex_cd,sex_long,sex_sort,circle_cd,circle_lang,circle_sort,quar_cd,quar_lang,quar_sort,race_1_text,race_2_text,breed_mixed__breed_cd,breed_mongrel_long,breed_mixed__breed_sort,breed_type_cd,breed_type_long,breed__type_sort,birth_dog_year,age_v_dog_cd,age_v_dog_long,age_v_dog_sort,sex_dog_cd,sex_dog_long,sex_dog_sort,dog_color_text,number_of_dogs
34428,2019,D,137678,30,30- bis 39-Jährige,4,2,weiblich,2,9,Kreis 9,9,91,Albisrieden,91,Malteser,Keine,1,Rassehund,1,K,Kleinwüchsig,1,2017,1,1-Jährige,1,1,männlich,1,weiss,1
9535,2016,D,92944,30,30- bis 39-Jährige,4,2,weiblich,2,11,Kreis 11,11,115,Oerlikon,115,Pudel,Keine,1,Rassehund,1,K,Kleinwüchsig,1,2009,6,6-Jährige,6,2,weiblich,2,silber,1
16247,2017,D,92851,50,50- bis 59-Jährige,6,2,weiblich,2,10,Kreis 10,10,102,Wipkingen,102,Jack Russel Terrier,Unbekannt,3,"Mischling, sekundäre Rasse unbekannt",3,K,Kleinwüchsig,1,2007,9,9-Jährige,9,2,weiblich,2,grau/schwarz,1


#### Zurich Population Dataset



In [6]:
# zurich_pop_link = "https://data.stadt-zuerich.ch/dataset/bev_bestand_jahr_quartier_alter_herkunft_geschlecht_od3903/download/BEV390OD3903.csv"
zurich_pop_link = "https://storage.googleapis.com/mrprime_dataset/zurich/zurich_pop.csv"
zurich_pop_data = hf.InfoDataFrame(pd.read_csv(zurich_pop_link))
zurich_pop_data.limit_info()
zurich_pop_data = hf.sanitize_df_column_names(zurich_pop_data)
zurich_pop_data.limit_info()
print("Showing a full row of the Zurich population DataFrame:")
zurich_pop_data.sample().T


Total number of columns: 23
<class 'helper_functions.InfoDataFrame'>
RangeIndex: 370658 entries, 0 to 370657
Columns: 23 entries, StichtagDatJahr to AnzBestWir
dtypes: int64(15), object(8)
memory usage: 65.0+ MB

Only showing info for 8 columns, chosen at random.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 370658 entries, 0 to 370657
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   SexLang       370658 non-null  object
 1   AlterV20Kurz  370658 non-null  object
 2   SexCd         370658 non-null  int64 
 3   AlterV05Kurz  370658 non-null  object
 4   KreisCd       370658 non-null  int64 
 5   SexKurz       370658 non-null  object
 6   QuarCd        370658 non-null  int64 
 7   HerkunftLang  370658 non-null  object
dtypes: int64(3), object(5)
memory usage: 22.6+ MB

Total number of columns: 23
<class 'helper_functions.InfoDataFrame'>
RangeIndex: 370658 entries, 0 to 370657
Columns: 23 entries, deadline_date

,258864
deadline_date_year,2014
age_v_sort,10
age_v_cd,10
age_v_short,10
age_v_05_sort,3
age_v_05_cd,10
age_v_05_short,10-14
age_v_10_cd,10
age_v_10_short,10-19
age_v_20_cd,0


#### Zurich Income Dataset
These data contain quantile values of the taxable income of natural persons who are primarily taxable in the city of Zurich. Tax income are in thousand francs (integer).

In [7]:
# zurich_income_link = "https://data.stadt-zuerich.ch/dataset/fd_median_einkommen_quartier_od1003/download/WIR100OD1003.csv"
zurich_income_link = (
    "https://storage.googleapis.com/mrprime_dataset/zurich/zurich_income.csv"
)
zurich_income_data = hf.InfoDataFrame(pd.read_csv(zurich_income_link))
zurich_income_data.info()

# Clean column names, display info and sample
zurich_income_data = hf.sanitize_df_column_names(zurich_income_data)


zurich_income_data.info()
print("\nShowing a full row of the Zurich income DataFrame:")
zurich_income_data.sample().T

<class 'helper_functions.InfoDataFrame'>
RangeIndex: 2244 entries, 0 to 2243
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   StichtagDatJahr      2244 non-null   int64  
 1   QuarSort             2244 non-null   int64  
 2   QuarCd               2244 non-null   int64  
 3   QuarLang             2244 non-null   object 
 4   SteuerTarifSort      2244 non-null   int64  
 5   SteuerTarifCd        2244 non-null   int64  
 6   SteuerTarifLang      2244 non-null   object 
 7   SteuerEinkommen_p50  2181 non-null   float64
 8   SteuerEinkommen_p25  2181 non-null   float64
 9   SteuerEinkommen_p75  2181 non-null   float64
dtypes: float64(3), int64(5), object(2)
memory usage: 175.4+ KB
<class 'helper_functions.InfoDataFrame'>
RangeIndex: 2244 entries, 0 to 2243
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   deadline_date

,888
deadline_date_year,2007
quar_sort,91
quar_cd,91
quar_lang,Albisrieden
tax_tariff_sort,0
tax_tariff_cd,0
tax_tariff_long,Grundtarif
tax_income_p_50,39.3
tax_income_p_25,21.0
tax_income_p_75,59.3


#### Zurich Household Dataset 

In [8]:
# zurich_household_data_link = "https://data.stadt-zuerich.ch/dataset/bev_hh_haushaltsgroesse_quartier_seit2013_od3806/download/BEV380OD3806.csv"
zurich_household_data_link = (
    "https://storage.googleapis.com/mrprime_dataset/zurich/zurich_household.csv"
)
zurich_household_data = hf.InfoDataFrame(
    pd.read_csv(zurich_household_data_link))
zurich_household_data.limit_info()

zurich_household_data = hf.sanitize_df_column_names(zurich_household_data)
zurich_household_data.limit_info()
print("\nShowing a full row of the Zurich household DataFrame:")

zurich_household_data.sample().T


Total number of columns: 9
<class 'helper_functions.InfoDataFrame'>
RangeIndex: 2040 entries, 0 to 2039
Columns: 9 entries, StichTagDatJahr to AnzBestWir
dtypes: int64(6), object(3)
memory usage: 143.6+ KB

Only showing info for 8 columns, chosen at random.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2040 entries, 0 to 2039
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   hh_groesseLang   2040 non-null   object
 1   AnzBestWir       2040 non-null   int64 
 2   hh_groesseSort   2040 non-null   int64 
 3   StichTagDatJahr  2040 non-null   int64 
 4   QuarSort         2040 non-null   int64 
 5   QuarLang         2040 non-null   object
 6   KreisSort        2040 non-null   int64 
 7   AnzHH            2040 non-null   int64 
dtypes: int64(6), object(2)
memory usage: 127.6+ KB

Total number of columns: 9
<class 'helper_functions.InfoDataFrame'>
RangeIndex: 2040 entries, 0 to 2039
Columns: 9 entries, key_day_

,1280
key_day_dat_year,2019
quar_sort,34
quar_lang,Sihlfeld
circle_sort,3
circle_lang,Kreis 3
hh_size_sort,3
hh_size_lang,3 Personen
number_hh,1245
number_we,3735


### Dataset Wrangling

Before diving into Exploratory Data Analysis (EDA), we need to prepare our datasets. This involves:
- Removing unnecessary columns
- Renaming columns for consistency
- Adding new columns
- Cleaning data (handling missing values, correcting datatypes, and standardizing data)

These steps will ensure our data is clean and well-structured, setting the stage for effective and accurate analysis in the EDA phase. We'll apply these steps to each dataset.

#### Zurich Statistical Districts Geospatial Data

Additional steps for this dataset not yet mentioned:

- area calculations
- spatial join with the geospatial data so that we can consider the districts if we wanted to

In [9]:
zurich_map_gdf = z_gdf["z_gdf_1"]

zurich_map_gdf.rename(
    columns={"qname": "neighborhood",
             "qnr": "sub_district", "knr": "district"},
    inplace=True,
)
# Format the sub_district column to have 3 digits
zurich_map_gdf["sub_district"] = zurich_map_gdf["sub_district"].astype(
    str).str.zfill(3)

# Create the refined geodataframe
neighborhood_gdf = zurich_map_gdf[
    ["neighborhood", "sub_district", "district", "geometry"]
].copy()

# Display geodataframe information and CRS
neighborhood_gdf.info()
display(neighborhood_gdf.crs)

# Display a sample entry from the transformed geodataframe
neighborhood_gdf.sample().T
# Load the geospatial data for calculation
zurich_calc_gdf = z_gdf["z_gdf_2"]

# Calculate area in square meters and add as a new column
zurich_calc_gdf["subd_area_km2"] = (
    zurich_calc_gdf.to_crs(ccrs.GOOGLE_MERCATOR).area / 1e6
)

# Rename the column for consistency with the main geodataframe
zurich_calc_gdf = zurich_calc_gdf.rename(columns={"qname": "neighborhood"})

# Merge calculated features with the main geodataframe (neighborhood_gdf)
area_gdf = neighborhood_gdf.merge(
    zurich_calc_gdf[["neighborhood", "subd_area_km2"]], on="neighborhood"
)

# Display a snapshot of the merged geodataframe
display(area_gdf.sample().T)


districts_gdf = (
    neighborhood_gdf.drop(columns=["neighborhood", "sub_district"])
    .dissolve(by="district")
    .reset_index()
)
districts_gdf = districts_gdf.dissolve(by="district").reset_index()
districts_gdf["d_area_km2"] = districts_gdf.to_crs(
    ccrs.GOOGLE_MERCATOR).area / 1e6

display(districts_gdf.sample().T)
districts_gdf

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 34 entries, 0 to 33
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   neighborhood  34 non-null     object  
 1   sub_district  34 non-null     object  
 2   district      34 non-null     int64   
 3   geometry      34 non-null     geometry
dtypes: geometry(1), int64(1), object(2)
memory usage: 1.2+ KB


<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- undefined
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

,29
neighborhood,Hochschulen
sub_district,012
district,1
geometry,"POLYGON ((8.5507374974 47.3723392838, 8.550656..."
subd_area_km2,1.266849


,5
district,6
geometry,"POLYGON ((8.5479683256 47.3991486798, 8.548007..."
d_area_km2,11.124162


,district,geometry,d_area_km2
0,1,"POLYGON ((8.54195 47.37971, 8.54196 47.37972, ...",3.922455
1,2,"POLYGON ((8.51982 47.32401, 8.51974 47.32401, ...",24.085868
2,3,"POLYGON ((8.51943 47.35125, 8.51889 47.35111, ...",18.841344
3,4,"POLYGON ((8.53301 47.37394, 8.53299 47.37392, ...",6.333365
4,5,"POLYGON ((8.52834 47.38939, 8.52862 47.38919, ...",4.353097
5,6,"POLYGON ((8.54797 47.39915, 8.54801 47.39918, ...",11.124162
6,7,"POLYGON ((8.60185 47.37186, 8.60188 47.37178, ...",32.719294
7,8,"POLYGON ((8.56493 47.34636, 8.56458 47.34619, ...",10.466348
8,9,"POLYGON ((8.50127 47.37961, 8.50121 47.37957, ...",26.281411
9,10,"POLYGON ((8.52545 47.40667, 8.52574 47.40675, ...",19.822487


In [10]:
# Save the geodataframe to disk in the data folder
area_gdf.to_file("../data/zurich_neighborhoods.geojson")
districts_gdf.to_file("../data/zurich_districts.geojson")

#### Zurich Dogs Dataset

The original dataset had 31 columns, many redundant. We've picked 18 for our analysis:

- deadline_date_year
- holder_id
- age_v_10_cd
- sex_cd
- circle_cd
- quar_cd
- quar_lang
- race_1_text
- race_2_text
- breed_mixed__breed_cd
- breed_mongrel_long
- breed_mixed__breed_sort
- breed_type_cd
- birth_dog_year
- age_v_dog_cd
- sex_dog_cd
- dog_color_text
- number_of_dogs

From these columns we create a new dataset, `dog_data` and we and we transform these column in preparation for the EDA phase. These transformations included:
- Converting the columns which only contain two different values two binary columns
- translating some values from German to English
- dealing with missing values
- standardizing some of the values for easier grouping.

In [51]:
new_column_names = {
    "deadline_date_year": "roster",
    "holder_id": "owner_id",
    "age_v_10_cd": "age_group_10",
    "sex_cd": "owner_gender",
    "breed_type_cd": "dog_size",
    "age_v_dog_cd": "dog_age",
    "breed_mongrel_long": "mixed_type",
    "sex_dog_cd": "dog_gender",
    "dog_color_text": "dog_color",
    "race_1_text": "breed_1",
    "race_2_text": "breed_2",
    "circle_cd": "district",
    "quar_cd": "sub_district",
}

zurich_dog_data = zurich_dog_data.rename(columns=new_column_names)

# After renaming, you may still need to adjust the data types for certain columns
zurich_dog_data["owner_id"] = zurich_dog_data["owner_id"].astype("string").str.zfill(6)
zurich_dog_data["dog_age"] = zurich_dog_data["dog_age"].astype(int)
zurich_dog_data["district"] = zurich_dog_data["district"].astype(int)
zurich_dog_data["sub_district"] = (
    zurich_dog_data["sub_district"].astype("string").str.zfill(3)
)
# Repeat each row based on the number of dogs in the row represents
zurich_dog_data = zurich_dog_data.loc[
    zurich_dog_data.index.repeat(zurich_dog_data["number_of_dogs"])
]
# drop the number of dogs column
zurich_dog_data.drop("number_of_dogs", axis=1, inplace=True)
# reset the index
zurich_dog_data.reset_index(drop=True, inplace=True)

print(
    f"Dataset now has {zurich_dog_data.shape[0]} rows and {zurich_dog_data.shape[1]} columns"
)

dog_data = zurich_dog_data[list(new_column_names.values())].copy()
display(
    dog_data.describe(include="all")
    .T.sort_values(by="unique")
    .infer_objects(copy=False)
    .fillna("")
)

Dataset now has 71212 rows and 31 columns


,count,unique,top,freq,mean,std,min,25%,50%,75%,max
dog_size,71212.0,4.0,K,43841.0,,,,,,,
mixed_type,71212.0,4.0,Rassehund,50926.0,,,,,,,
sub_district,71212.0,40.0,092,5556.0,,,,,,,
breed_2,71212.0,176.0,Keine,50926.0,,,,,,,
dog_color,71212.0,214.0,schwarz,7547.0,,,,,,,
breed_1,71212.0,394.0,Unbekannt,9109.0,,,,,,,
owner_id,71212.0,15504.0,105585,109.0,,,,,,,
roster,71212.0,,,,2019.282761,2.599665,2015.0,2017.0,2019.0,2022.0,2023.0
age_group_10,71212.0,,,,47.817545,56.048544,10.0,30.0,40.0,60.0,999.0
owner_gender,71212.0,,,,1.690108,0.462452,1.0,1.0,2.0,2.0,2.0


In [52]:
# Unique values for "mixed_type" column
breed_cat_list_de = zurich_dog_data["mixed_type"].unique().tolist()
print("Breed Categories (German):")
display(breed_cat_list_de)

# Create a dictionary for translation
breed_cat_dict = translate_list_to_dict(breed_cat_list_de)
print("\nBreed Category Dictionary (Translation):")
display(breed_cat_dict)

Breed Categories (German):


['Rassehund',
 'Mischling, beide Rassen bekannt',
 'Mischling, sekundäre Rasse unbekannt',
 'Mischling, beide Rassen unbekannt']


Breed Category Dictionary (Translation):


{'Rassehund': 'Pedigree dog',
 'Mischling, beide Rassen bekannt': 'Mixed breed, both breeds known',
 'Mischling, sekundäre Rasse unbekannt': 'Mixed breed, secondary breed unknown',
 'Mischling, beide Rassen unbekannt': 'Mixed breed, both breeds unknown'}

In [53]:
# Map 'mixed_type' to categories, rename for brevity, and define 'is_pure_breed'
dog_data["mixed_type"] = (
    dog_data["mixed_type"]
    .map(breed_cat_dict)
    .map(
        {
            "Pedigree dog": "PB",
            "Mixed breed, both breeds known": "BB",
            "Mixed breed, secondary breed unknown": "BU",
            "Mixed breed, both breeds unknown": "UU",
        }
    )
)
dog_data["is_pure_breed"] = dog_data["mixed_type"].eq("PB")

In [54]:
# Define owner and dog gender
dog_data["is_male_owner"] = dog_data["owner_gender"] == 1
dog_data["is_male_dog"] = dog_data["dog_gender"] == 1

# Drop the columns we just used to create the new columns
dog_data.drop(columns=["owner_gender", "dog_gender"], inplace=True)

In [55]:
# Unique values for dog colors
dog_colors = dog_data["dog_color"].str.lower().unique()

# Translate dog colors
dog_color_translations = translate_list_to_dict(dog_colors)
dog_data["dog_color_en"] = dog_data["dog_color"].str.lower().map(dog_color_translations)

# Unique values for breed_1
breeds_1 = dog_data["breed_1"].str.lower().unique()

# Translate breed_1
breed_1_translations = translate_list_to_dict(breeds_1)
dog_data["breed_1_en"] = dog_data["breed_1"].str.lower().map(breed_1_translations)

# Unique values for breed_2
breeds_2 = dog_data["breed_2"].str.lower().unique()

# Translate breed_2
breed_2_translations = translate_list_to_dict(breeds_2)
dog_data["breed_2_en"] = dog_data["breed_2"].str.lower().map(breed_2_translations)


##### Breed Standardization
To ensure consistency in the analysis, the breeds in the dataset are standardized. Since the "breed" column is free text, allowing dog owners to input their breed information during registration, variations can exist even for the same breeds. To address this, we will use the dataframe we collected in the last notebook which contains the breeds recognized by the FCI (Fédération Cynologique Internationale). Within this dataframe, each recognized FCI breed has a column listing its name in different languages and alternative, unofficial names. 

This approach helps capture variations in breed names and facilitates grouping similar breeds together.




In [56]:
# Get the FCI dataframe with the recognized breeds
fci_breeds = pd.read_json("../data/fci_breeds.json")
fci_breeds[["alt_names", "breed_en"]]

# Create a DataFrame with translated breed names
breeds_df = pd.DataFrame.from_dict(
    {**breed_1_translations, **breed_2_translations}, orient="index"
).reset_index()
breeds_df.columns = ["breed_de", "breed_en"]

# Initialize a "standard" column for breed standardization
breeds_df["standard"] = None
nan_mask = breeds_df["standard"].isna()

# Match each column for breed standardization
for col in breeds_df.columns:
    matched_value = hf.apply_fuzzy_matching_to_breed_column(
        breeds_df.loc[nan_mask], col, fci_breeds, [fuzz.WRatio]
    )
    breeds_df.loc[nan_mask, "standard"] = matched_value[nan_mask]
    nan_mask = breeds_df["standard"].isna()

# Update the standard column for specific cases
breeds_df.loc[nan_mask, "standard"] = breeds_df.loc[nan_mask, "breed_en"]
breeds_df.loc[breeds_df["breed_de"] == "elo", "standard"] = "elo"
breeds_df.loc[breeds_df["breed_de"] == "keine", "standard"] = "none"
breeds_df.loc[breeds_df["breed_de"] == "mischling", "standard"] = "hybrid"

# Convert breed_1 to lowercase for merging
dog_data["breed_1"] = dog_data["breed_1"].str.lower()
dog_data["breed_2"] = dog_data["breed_2"].str.lower()

# Merge with the breeds_df for standardized breed names
dog_data = dog_data.merge(
    breeds_df.drop(columns=["breed_en"]),
    left_on="breed_1",
    right_on="breed_de",
    suffixes=("", "_1"),
)

dog_data = dog_data.merge(
    breeds_df.drop(columns=["breed_en"]),
    left_on="breed_2",
    right_on="breed_de",
    suffixes=("", "_2"),  # Add suffix to distinguish columns
)


##### Filtering Doodle Dogs

A specific analysis is conducted to filter out dogs with 'doodle' in their breed names, converting them to mixed breeds and updating breed information accordingly. This is a designer breed which is not yet recognized.


In [57]:
# Create mask to filter out the doodle dogs
doodle_mask = dog_data["breed_1"].str.contains(
    r".*doodle", regex=True, na=False, case=False
)
print(f"Number of doodle dogs: {doodle_mask.sum()}")
# convert them to mixed breed if they are pure breeds
dog_data.loc[doodle_mask, "is_pure_breed"] = False
dog_data.loc[doodle_mask, "breed_2"] = "Pudel"
dog_data.loc[doodle_mask, "mixed_type"] = "BB"
dog_data.loc[doodle_mask, "breed_1"] = dog_data.loc[doodle_mask, "breed_1"].apply(
    lambda x: "Golden Retriever" if x.startswith("G") else "Labrador Retriever"
)
# dog_data[doodle_mask]

Number of doodle dogs: 27



The number of dogs for each row is given in the `number_of_dogs` column. These are 'brothers and sisters' that also have the same owner and same characteristics.

E.g.
- `standard` or breed 
- `dog_color_en` or dog color, etc. 


We expand the dataset to have one dog for each row, by repeating the rows by the number in the `number_of_dogs` column. We reset the index after so that we have a unique index for each row.


In [58]:
# Calculate total dogs per owner and roster
dog_data["pet_count"] = dog_data.groupby(["owner_id", "roster"])["breed_1"].transform(
    "count"
)

##### Missing Values
Although initially it looked as if we have no missing values, on close investigation we can see that there are placeholder values for where the missing values are. We replaced these with `Nan` values so that they are not mistaken for real values. As these are only few for the columns `sub_district`, `dog_age`, and `district` we simply drop those rows. Remaining column with missing values `age_group_10`, we:

- fill missing `age_group_10` (dog owners' age groups) with `-1`, tracking these in `age_group_missing`.
- use later years' rosters to fill age group where possible, and make these edits in `age_group_10`.


Finally, we create `age_group_20`, grouping ages into 20-year increments, approximating a generation's length. 


In [59]:
display(
    dog_data.describe(include="all")
    .T.sort_values(by="unique")
    .infer_objects(copy=False)
    .fillna("")
)

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
is_male_owner,71212.0,2.0,False,49144.0,,,,,,,
is_male_dog,71212.0,2.0,False,35709.0,,,,,,,
is_pure_breed,71212.0,2.0,True,50902.0,,,,,,,
dog_size,71212.0,4.0,K,43841.0,,,,,,,
mixed_type,71212.0,4.0,PB,50902.0,,,,,,,
sub_district,71212.0,40.0,092,5556.0,,,,,,,
standard_2,71212.0,133.0,none,50926.0,,,,,,,
breed_2_en,71212.0,173.0,no,50926.0,,,,,,,
breed_de_2,71212.0,176.0,keine,50926.0,,,,,,,
breed_2,71212.0,177.0,keine,50902.0,,,,,,,


In [60]:
# Create a list of sub_districts to be used for validation
sub_districts_list = neighborhood_gdf["sub_district"].unique().tolist()

# Define a dictionary of conditions and corresponding columns to be updated
conditions = {
    "dog_size": dog_data["dog_size"] == "UN",
    "age_group_10": dog_data["age_group_10"] > 100,
    "district": dog_data["district"] > 12,
    "dog_age": dog_data["dog_age"] > 30,
    "sub_district": ~dog_data["sub_district"].isin(sub_districts_list),
}

# Identify and print unique breeds with 'UN' dog size
un_breeds = dog_data.loc[conditions["dog_size"], "breed_1"].unique()
print(f"Dogs breeds of those missing dog_size data:\n{un_breeds}")

# Replace 'UN' dog size with 'K' and other invalid values with NaN
for column, condition in conditions.items():
    dog_data.loc[condition, column] = "K" if column == "dog_size" else np.nan

# Display the number of NaN values in each column
print("\nNumber of NaN values in each column:")
print(dog_data.isna().sum().sort_values(ascending=False))

Dogs breeds of those missing dog_size data:
['unbekannt' 'podengo portugues klein' 'mischling']

Number of NaN values in each column:
age_group_10     227
sub_district      18
dog_age            8
district           4
roster             0
is_male_dog        0
standard_2         0
breed_de_2         0
standard           0
breed_de           0
breed_2_en         0
breed_1_en         0
dog_color_en       0
is_pure_breed      0
is_male_owner      0
owner_id           0
breed_2            0
breed_1            0
dog_color          0
mixed_type         0
dog_size           0
pet_count          0
dtype: int64


In [61]:
dog_data.dropna(subset=["dog_age", "district", "sub_district"], inplace=True)

In [62]:
# convert the numerical columns which had NaN values to int
dog_data["dog_age"] = dog_data["dog_age"].astype(int)
dog_data["district"] = dog_data["district"].astype(int)

In [63]:
# Create an indicator variable for missing 'age_group_10' values
dog_data["age_group_missing"] = dog_data["age_group_10"].isna().astype(int)

# Fill in the missing 'age_group_10' values
dog_data["age_group_10"] = dog_data["age_group_10"].fillna(
    dog_data.groupby("owner_id")["age_group_10"].transform(
        lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan
    )
)

dog_data["age_group_10"] = dog_data["age_group_10"].fillna(-1).astype(int)
dog_data["age_group_20"] = dog_data["age_group_10"].apply(
    lambda x: -1 if x == -1 else (x // 20) * 20
)

In [64]:
dog_data.sample(3)

,roster,owner_id,age_group_10,dog_size,dog_age,mixed_type,dog_color,breed_1,breed_2,district,sub_district,is_pure_breed,is_male_owner,is_male_dog,dog_color_en,breed_1_en,breed_2_en,breed_de,standard,breed_de_2,standard_2,pet_count,age_group_missing,age_group_20
37561,2020,089094,50,K,14,PB,rot/weiss,parson jack russell terrier,keine,9,092,True,True,True,Red White,parson jack russell terrier,no,parson jack russell terrier,jack russell terrier,keine,none,1,0,40
62632,2023,087375,60,I,2,BB,beige,labrador retriever,pudel,7,074,False,False,True,beige,labrador retrievers,poodle,labrador retriever,labrador retriever,pudel,poodle,1,0,60
16546,2017,095765,50,I,8,BB,schwarz,labrador retriever,boxer,10,102,False,True,True,black,labrador retrievers,boxer,labrador retriever,labrador retriever,boxer,boxer,1,0,40


##### Consolidated Dog Data preprocessing
Combined all that we did with the dog data set into the `preprocess_dog_data` function.

In [11]:
fci_breeds = pd.read_json("../data/fci_breeds.json")
fci_breeds[["alt_names", "breed_en"]]


def get_translation_dict(data, column):
    """Returns a dataframe with the unique values in the column and their translations"""
    df = data.copy()
    data_to_translate = df[column].str.lower().unique()

    return translate_list_to_dict(data_to_translate)


def get_breed_standard(data, column, agency_breeds_df=fci_breeds):
    """Find the breed standard for each breed in the column"""
    df = data.copy()
    translated_dict = get_translation_dict(df, column)
    # convert the dict to a dataframe
    translated_df = pd.DataFrame.from_dict(
        translated_dict, orient="index"
    ).reset_index()
    translated_df.columns = [f"{column}_de", f"{column}_en"]

    # apply fuzzy matching to the breed column to get the standardized breed name
    # create the 'standard' column and fill it with None
    translated_df["standard"] = None
    nan_mask = translated_df["standard"].isna()
    # Match each column for breed standardization

    for col in translated_df.columns:
        matched_value = hf.apply_fuzzy_matching_to_breed_column(
            translated_df.loc[nan_mask], col, agency_breeds_df, [fuzz.WRatio]
        )
        translated_df.loc[nan_mask, "standard"] = matched_value[nan_mask]
        nan_mask = translated_df["standard"].isna()

    # Update the standard column for specific cases
    translated_df.loc[nan_mask, "standard"] = translated_df.loc[
        nan_mask, f"{column}_en"
    ]
    translated_df.loc[translated_df[f"{column}_de"] == "elo", "standard"] = "elo"
    translated_df.loc[translated_df[f"{column}_de"] == "keine", "standard"] = "none"
    translated_df.loc[
        translated_df[f"{column}_de"] == "mischling", "standard"
    ] = "hybrid"
    return translated_df


def get_doodle_fix(data):
    """Correct doodle dogs to standard entries"""
    df = data.copy()
    # Create mask to filter out the doodle dogs
    doodle_mask = df["breed_1_de"].str.contains(
        r".*doodle", regex=True, na=False, case=False
    )
    # convert them to mixed breed if they are pure breeds
    df.loc[doodle_mask, "is_pure_breed"] = False
    df.loc[doodle_mask, "breed_2_de"] = "Pudel"
    df.loc[doodle_mask, "mixed_type"] = "BB"
    df.loc[doodle_mask, "breed_1_de"] = df.loc[doodle_mask, "breed_1_de"].apply(
        lambda x: "Golden Retriever" if x.startswith("G") else "Labrador Retriever"
    )
    return df


def drop_concealed_nans(data):
    df = data.copy()
    sub_districts_list = df.sub_district.value_counts().index.tolist()[:34]
    nan_conditions = {
        "dog_size": df["dog_size"] == "UN",
        "age_group_10": df["age_group_10"] > 100,
        "district": df["district"] > 12,
        "dog_age": df["dog_age"] > 30,
        "sub_district": ~df["sub_district"].isin(sub_districts_list),
    }
    for column, condition in nan_conditions.items():
        df.loc[condition, column] = "K" if column == "dog_size" else np.nan
    return df


# define a function which does all of the preprocessing steps
def preprocess_dog_data(data, **kwargs):
    """Preprocess the Zurich dog data"""
    new_column_names = {
        "deadline_date_year": "roster",
        "holder_id": "owner_id",
        "age_v_10_cd": "age_group_10",
        "sex_cd": "owner_gender",
        "breed_type_cd": "dog_size",
        "age_v_dog_cd": "dog_age",
        "breed_mongrel_long": "mixed_type",
        "sex_dog_cd": "dog_gender",
        "dog_color_text": "dog_color",
        "race_1_text": "breed_1",
        "race_2_text": "breed_2",
        "circle_cd": "district",
        "quar_cd": "sub_district",
    }
    breed_cat_dict = {
        "Rassehund": "Pedigree dog",
        "Mischling, beide Rassen bekannt": "Mixed breed, both breeds known",
        "Mischling, sekundäre Rasse unbekannt": "Mixed breed, secondary breed unknown",
        "Mischling, beide Rassen unbekannt": "Mixed breed, both breeds unknown",
    }
    breed_cat_abv_dict = {
        "Pedigree dog": "PB",
        "Mixed breed, both breeds known": "BB",
        "Mixed breed, secondary breed unknown": "BU",
        "Mixed breed, both breeds unknown": "UU",
    }

    df = data.copy()
    df = df.rename(columns=new_column_names)
    df["owner_id"] = df["owner_id"].astype("string").str.zfill(6)
    df["dog_age"] = df["dog_age"].astype(int)
    df["district"] = df["district"].astype(int)
    df["sub_district"] = df["sub_district"].astype("string").str.zfill(3)
    df = df.loc[df.index.repeat(df["number_of_dogs"])]
    df = df.drop("number_of_dogs", axis=1)
    df = df.reset_index(drop=True)
    df["mixed_type"] = df["mixed_type"].map(breed_cat_dict).map(breed_cat_abv_dict)
    df["is_pure_breed"] = df["mixed_type"].eq("PB")
    df["is_male_owner"] = df["owner_gender"] == 1
    df["is_male_dog"] = df["dog_gender"] == 1
    df = df.drop(columns=["owner_gender", "dog_gender"])
    df["dog_color_en"] = (
        df["dog_color"].str.lower().map(get_translation_dict(df, "dog_color"))
    )
    df["breed_1_de"] = df["breed_1"].str.lower()
    df["breed_2_de"] = df["breed_2"].str.lower()
    df = df.merge(
        get_breed_standard(df, "breed_1"),
        left_on="breed_1_de",
        right_on="breed_1_de",
        suffixes=("", "_1"),
    )
    df = df.merge(
        get_breed_standard(df, "breed_2"),
        left_on="breed_2_de",
        right_on="breed_2_de",
        suffixes=("", "_2"),
    )
    df = get_doodle_fix(df)
    df = drop_concealed_nans(df)

    df = df.dropna(subset=["dog_age", "district", "sub_district"])
    df["dog_age"] = df["dog_age"].astype(int)
    df["district"] = df["district"].astype(int)
    df["pet_count"] = df.groupby(["owner_id", "roster"])["breed_1"].transform("count")
    df["age_group_missing"] = df["age_group_10"].isna().astype(int)
    df["age_group_10"] = df["age_group_10"].fillna(
        df.groupby("owner_id")["age_group_10"].transform(
            lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan
        )
    )
    df["age_group_10"] = df["age_group_10"].fillna(-1).astype(int)
    df["age_group_20"] = df["age_group_10"].apply(
        lambda x: -1 if x == -1 else (x // 20) * 20
    )

    return df

In [14]:
dog_data_columns_to_keep = [
    "roster",
    "owner_id",
    "dog_size",
    "dog_age",
    "age_group_10",
    "age_group_20",
    "mixed_type",
    "is_pure_breed",
    "is_male_owner",
    "is_male_dog",
    "dog_color_en",
    "standard",
    "standard_2",
    "pet_count",
    "district",
    "sub_district",
    "age_group_missing",
]

In [ ]:

# Align for the roster years from 2015-2022
dog_data = hf.query_for_time_period(dog_data)

# Save the processed dog data to disk
dog_data[dog_data_columns_to_keep].to_csv("../data/processed_dog_data.csv", index=False)

# filtered_dog_data.sample(5)

In [15]:
hf.query_for_time_period(
    zurich_dog_data, start_year=2015, end_year=2017, year_col="deadline_date_year"
)
# separate out the dog data for 2023 in a separate dataframe
zurich_dog_data_2023 = hf.query_for_time_period(
    zurich_dog_data, 2023, 2024, year_col="deadline_date_year")
# single function to preprocess the dog data
zurich_dog_data_2023 = preprocess_dog_data(zurich_dog_data_2023)
zurich_dog_data_2023[dog_data_columns_to_keep].to_csv(
    '../data/processed_dog_data_2023.csv')

#### Zurich Population Dataset

In [ ]:
# Create 'is_swiss' column, True if 'origin_lang' contains 'Schweizer'
zurich_pop_data["is_swiss"] = zurich_pop_data["origin_lang"].str.contains(
    "Schweizer", regex=False, na=False, case=False
)

# Create new columns with copied data for further processing
zurich_pop_data["neighborhood"] = zurich_pop_data["quar_lang"].copy()
zurich_pop_data["district"] = zurich_pop_data["circle_cd"].astype(int)

# Create 'sub_district' column, ensuring it's a string with leading zeros
zurich_pop_data["sub_district"] = (
    zurich_pop_data["quar_cd"].astype("string").str.zfill(3)
)

# Create new columns with copied data for further processing
zurich_pop_data["roster"] = zurich_pop_data["deadline_date_year"].copy()
zurich_pop_data["age_group_10"] = zurich_pop_data["age_v_10_cd"].copy()
zurich_pop_data["age_group_20"] = zurich_pop_data["age_v_20_cd"].copy()
zurich_pop_data["age"] = zurich_pop_data["age_v_cd"].copy()
zurich_pop_data["pop_count"] = zurich_pop_data["number_we"].copy()

# Create 'is_male' column, True if 'sex_cd' equals 1
zurich_pop_data["is_male"] = zurich_pop_data["sex_cd"] == 1

# only keep the row which have the same age_group_10 as the dog_dataset
# zurich_pop_data = zurich_pop_data.loc[
#     zurich_pop_data["age_group_10"].isin(dog_data["age_group_10"].unique())
# ]


# Define a list of columns of interest for the final dataframe
columns_of_interest_pop = [
    "is_swiss",
    "neighborhood",
    "district",
    "sub_district",
    "roster",
    "age_group_10",
    "age_group_20",
    "is_male",
    "pop_count",
]

# Create a new dataframe 'pop_data' with only the columns of interest
pop_data = zurich_pop_data[columns_of_interest_pop].copy()
# align the roster years with the dog_data
pop_data = hf.query_for_time_period(pop_data)

# Display the structure of 'pop_data' dataframe for verification
pop_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100718 entries, 269940 to 370657
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   is_swiss      100718 non-null  bool  
 1   neighborhood  100718 non-null  object
 2   district      100718 non-null  int32 
 3   sub_district  100718 non-null  string
 4   roster        100718 non-null  int64 
 5   age_group_10  100718 non-null  int64 
 6   age_group_20  100718 non-null  int64 
 7   is_male       100718 non-null  bool  
 8   pop_count     100718 non-null  int64 
dtypes: bool(2), int32(1), int64(4), object(1), string(1)
memory usage: 6.0+ MB


In [ ]:
# save the processed population data to data folder
pop_data.to_csv("../data/processed_pop_data.csv", index=False)

#### Zurich Income Dataset


In [ ]:
zurich_income_data

,deadline_date_year,quar_sort,quar_cd,quar_lang,tax_tariff_sort,tax_tariff_cd,tax_tariff_long,tax_income_p_50,tax_income_p_25,tax_income_p_75
0,1999,11,11,Rathaus,0,0,Grundtarif,39.90,18.7,65.20
1,1999,11,11,Rathaus,1,1,Verheiratetentarif,80.40,48.1,128.00
2,1999,11,11,Rathaus,2,2,Einelternfamilientarif,44.60,25.0,87.50
3,1999,12,12,Hochschulen,0,0,Grundtarif,31.05,12.4,55.20
4,1999,12,12,Hochschulen,1,1,Verheiratetentarif,89.20,52.0,178.70
...,...,...,...,...,...,...,...,...,...,...
2239,2020,122,122,Schwamendingen-Mitte,1,1,Verheiratetentarif,62.85,39.7,89.15
2240,2020,122,122,Schwamendingen-Mitte,2,2,Einelternfamilientarif,30.90,14.6,52.30
2241,2020,123,123,Hirzenbach,0,0,Grundtarif,33.50,13.5,52.30
2242,2020,123,123,Hirzenbach,1,1,Verheiratetentarif,64.45,40.6,88.00


In [ ]:
# Extract unique values from 'tax_tariff_long' column and convert to list
tax_tariff_long_de = zurich_income_data.tax_tariff_long.unique().tolist()

# Translate the list to a dictionary using a helper function
tax_tariff_long_translated = translate_list_to_dict(tax_tariff_long_de)

# Display the translated dictionary for verification
display(tax_tariff_long_translated)

# Map the translated dictionary to 'tax_tariff_long' column, creating a new 'tax_status' column
zurich_income_data["tax_status"] = zurich_income_data.tax_tariff_long.map(
    tax_tariff_long_translated
)


# Create a dictionary mapping old column names to new ones
income_data_column_mapping = {
    "quar_lang": "neighborhood",
    "deadline_date_year": "roster",
    "tax_income_p_50": "median_income",
    "tax_income_p_25": "lower_q_income",
    "tax_income_p_75": "upper_q_income",
}

# Rename the columns
zurich_income_data = zurich_income_data.rename(columns=income_data_column_mapping)

# Perform the other transformations
zurich_income_data["tax_status"] = zurich_income_data.tax_tariff_long.map(
    tax_tariff_long_translated
)
zurich_income_data["sub_district"] = (
    zurich_income_data["quar_cd"].astype(int).astype("string").str.zfill(3)
)
zurich_income_data["district"] = zurich_income_data["sub_district"].str[:2].astype(int)


# Define a list of columns of interest for the final dataframe
columns_of_interest_income = [
    "neighborhood",
    "roster",
    "district",
    "sub_district",
    "tax_status",
    "median_income",
    "lower_q_income",
    "upper_q_income",
]


display(
    zurich_income_data.describe(include="all")
    .T.sort_values(by="unique")
    .infer_objects(copy=False)
    .fillna("")
)

{'Grundtarif': 'Basic tariff',
 'Verheiratetentarif': 'Married rate',
 'Einelternfamilientarif': 'Single-parent family tariff'}

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
tax_tariff_long,2244.0,3.0,Grundtarif,748.0,,,,,,,
tax_status,2244.0,3.0,Basic tariff,748.0,,,,,,,
neighborhood,2244.0,34.0,Rathaus,66.0,,,,,,,
sub_district,2244.0,34.0,011,66.0,,,,,,,
roster,2244.0,,,,2009.5,6.345703,1999.0,2004.0,2009.5,2015.0,2020.0
quar_sort,2244.0,,,,64.794118,35.95387,11.0,33.0,67.0,92.0,123.0
quar_cd,2244.0,,,,64.794118,35.95387,11.0,33.0,67.0,92.0,123.0
tax_tariff_sort,2244.0,,,,1.0,0.816679,0.0,0.0,1.0,2.0,2.0
tax_tariff_cd,2244.0,,,,1.0,0.816679,0.0,0.0,1.0,2.0,2.0
median_income,2181.0,,,,59.327873,27.796075,24.65,39.9,51.1,68.8,172.9


##### Handling Missing Income Data

The income datasets only extend up to 2020 due to the tax data evaluation process. To fill in the missing data for 2021 and 2022, we explored 2 strategies:

- Using all available data from 1999 onwards
- Applying a log transformation to the data from 1999 onwards

We assessed these strategies using various metrics, including mean absolute error, mean absolute percentage error, median absolute error, mean squared error, mean squared logarithmic error, and R2 score.

We employed an auto ARIMA model to predict the values for 2021 and 2022. We tested the model's accuracy by predicting the values for 2019 and 2020 using the income data from 1999 to 2018 and the log-transformed income data from the same period.

The log-transformed data provided more accurate predictions, as indicated by lower mean absolute error, mean absolute percentage error, and median absolute percentage error. Therefore, we chose this approach to predict the missing income data for 2021 and 2022 for the 34 neighborhoods.

In [ ]:
income_from_1999 = (
    zurich_income_data[columns_of_interest_income]
    .groupby(["roster", "sub_district"])[
        ["median_income", "lower_q_income", "upper_q_income"]
    ]
    .median()
    .round(3)
    .reset_index()
)
income_from_1999

,roster,sub_district,median_income,lower_q_income,upper_q_income
0,1999,011,44.600,25.00,87.50
1,1999,012,60.125,32.20,116.95
2,1999,013,75.200,46.15,119.00
3,1999,014,50.550,28.60,78.20
4,1999,021,38.600,21.75,57.75
...,...,...,...,...,...
743,2020,115,57.750,33.35,87.25
744,2020,119,41.800,21.10,67.05
745,2020,121,39.700,19.50,57.30
746,2020,122,35.300,17.20,54.60


In [ ]:
def create_pivot(df, column):
    """Create a pivot table and a pivot table of the natural logarithm of the specified column."""
    df[f"lg_{column}"] = np.log(df[column])
    pivot = (
        df[["sub_district", "roster", column]]
        .pivot(index="roster", columns="sub_district", values=column)
        .asfreq("YS")
    )
    lg_pivot = (
        df[["sub_district", "roster", f"lg_{column}"]]
        .pivot(index="roster", columns="sub_district", values=f"lg_{column}")
        .asfreq("YS")
    )
    return pivot, lg_pivot


# Convert the 'roster' column to datetime format
income_from_1999["roster"] = pd.to_datetime(income_from_1999["roster"], format="%Y")

# Create pivot tables
median_income_pivot_from_1999, lg_median_income_pivot_from_1999 = create_pivot(
    income_from_1999, "median_income"
)
lower_q_income_pivot_from_1999, lg_lower_q_income_pivot_from_1999 = create_pivot(
    income_from_1999, "lower_q_income"
)
upper_q_income_pivot_from_1999, lg_upper_q_income_pivot_from_1999 = create_pivot(
    income_from_1999, "upper_q_income"
)

In [ ]:
print("Pivot table of the natural logarithm of the median income:")
display(lg_median_income_pivot_from_1999.tail())

print("\nPivot table of the median income:")
median_income_pivot_from_1999.tail()

Pivot table of the natural logarithm of the median income:


sub_district,011,012,013,014,021,023,024,031,033,034,041,042,044,051,052,061,063,071,072,073,074,081,082,083,091,092,101,102,111,115,119,121,122,123
roster,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-01-01,4.097672,4.632299,4.399375,4.169761,3.906407,3.698830,4.183576,4.069881,3.693867,3.763523,3.933784,3.854394,3.535145,3.875359,4.269697,3.960813,4.264087,4.495355,4.424847,4.174387,4.105944,4.370713,4.085136,4.050044,3.887730,3.708682,4.051785,3.945458,3.667400,3.956996,3.663562,3.563883,3.529297,3.407842
2017-01-01,4.130355,4.537961,4.319819,4.234107,3.878466,3.740048,4.220977,4.054390,3.723281,3.795489,3.925926,3.896909,3.563883,3.808882,4.237723,3.958907,4.237001,4.588024,4.423648,4.106767,4.084294,4.317488,4.092677,4.162003,3.946424,3.728100,4.000034,3.979682,3.681351,3.929863,3.668677,3.634951,3.555348,3.443618
2018-01-01,4.044804,4.493959,4.460433,4.186620,3.931826,3.754199,4.261270,4.133565,3.753027,3.806662,3.912023,3.914021,3.586293,3.822098,4.328098,4.007333,4.287716,4.553877,4.424248,4.147885,4.151040,4.439116,4.141546,4.318821,3.987130,3.765840,4.063885,4.012773,3.688879,4.014580,3.675034,3.621671,3.555348,3.487375
2019-01-01,4.100161,4.510860,4.419744,4.273536,3.969348,3.749504,4.312811,4.160444,3.775057,3.819908,3.957952,3.972177,3.655840,3.896909,4.393214,4.027136,4.284965,4.590057,4.419443,4.219508,4.178226,4.480740,4.212868,4.342506,4.005513,3.781914,4.051785,4.043928,3.716008,4.002777,3.688879,3.676301,3.558201,3.502550
2020-01-01,4.105944,4.455509,4.483850,4.287716,3.979682,3.777348,4.295924,4.174387,3.779634,3.822098,4.039536,4.036009,3.632309,3.887730,4.354141,4.039536,4.278747,4.521789,4.441474,4.219508,4.192680,4.423648,4.219508,4.290459,4.028917,3.777348,4.106767,4.067316,3.725693,4.056123,3.732896,3.681351,3.563883,3.546740



Pivot table of the median income:


sub_district,011,012,013,014,021,023,024,031,033,034,041,042,044,051,052,061,063,071,072,073,074,081,082,083,091,092,101,102,111,115,119,121,122,123
roster,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-01-01,60.20,102.750,81.400,64.700,49.72,40.4,65.60,58.55,40.20,43.1,51.10,47.20,34.3,48.20,71.50,52.5,71.10,89.6,83.50,65.00,60.70,79.1,59.45,57.4,48.80,40.8,57.50,51.70,39.15,52.30,39.00,35.3,34.1,30.2
2017-01-01,62.20,93.500,75.175,69.000,48.35,42.1,68.10,57.65,41.40,44.5,50.70,49.25,35.3,45.10,69.25,52.4,69.20,98.3,83.40,60.75,59.40,75.0,59.90,64.2,51.75,41.6,54.60,53.50,39.70,50.90,39.20,37.9,35.0,31.3
2018-01-01,57.10,89.475,86.525,65.800,51.00,42.7,70.90,62.40,42.65,45.0,50.00,50.10,36.1,45.70,75.80,55.0,72.80,95.0,83.45,63.30,63.50,84.7,62.90,75.1,53.90,43.2,58.20,55.30,40.00,55.40,39.45,37.4,35.0,32.7
2019-01-01,60.35,91.000,83.075,71.775,52.95,42.5,74.65,64.10,43.60,45.6,52.35,53.10,38.7,49.25,80.90,56.1,72.60,98.5,83.05,68.00,65.25,88.3,67.55,76.9,54.90,43.9,57.50,57.05,41.10,54.75,40.00,39.5,35.1,33.2
2020-01-01,60.70,86.100,88.575,72.800,53.50,43.7,73.40,65.00,43.80,45.7,56.80,56.60,37.8,48.80,77.80,56.8,72.15,92.0,84.90,68.00,66.20,83.4,68.00,73.0,56.20,43.7,60.75,58.40,41.50,57.75,41.80,39.7,35.3,34.7


In [ ]:
my_metrics = [
    metrics.mean_absolute_error,
    metrics.mean_absolute_percentage_error,
    metrics.median_absolute_error,
    metrics.mean_squared_error,
    metrics.mean_squared_log_error,
    metrics.r2_score,
]


def calculate_metrics(actual, predicted):
    actual_values = actual.loc[predicted.index].values.ravel()
    predicted_values = predicted.values.ravel()
    return {
        metric.__name__: metric(actual_values, predicted_values)
        for metric in my_metrics
    }


calculate_metrics_partial = partial(calculate_metrics, median_income_pivot_from_1999)


def convert_to_long_format(df, value_name="value"):
    """Converts a DataFrame from wide to long format."""
    return (
        df.unstack()
        .reset_index()
        .rename(columns={"level_0": "sub_district", "level_1": "roster", 0: value_name})
    )


def plot_arima_forecast(long_df, value_name, vline=2020, **kwargs):
    """Plots the value_name column of a Dataframe in long format."""
    forecast_color = kwargs.get("forecast_color", "gray")
    non_forecast_color = kwargs.get("non_forecast_color", "gray")

    v_line = hv.VLine(x=pd.to_datetime(f"{vline}")).opts(
        color="red", line_dash="dotted"
    )
    forecast_df = long_df.loc[long_df["roster"] >= f"{vline}"]
    forecast_line = forecast_df.hvplot(
        x="roster",
        y=value_name,
        by="sub_district",
        color=forecast_color,
        line_dash="dashed",
    )
    non_forecast_df = long_df.loc[long_df["roster"] <= f"{vline}"]
    non_forecast_line = non_forecast_df.hvplot(
        x="roster", y=value_name, by="sub_district", color=non_forecast_color
    )
    return non_forecast_line * forecast_line * v_line

In [ ]:
# Arima models to assess the forecast of the median income using log values
lg_from_1999_pred_last_2 = hf.forecast_arima(
    lg_median_income_pivot_from_1999, 2019, n_periods=2, model_desc="Log Model 1999"
)
# Arima models to assess the forecast of the median income
from_1999_pred_last_2 = hf.forecast_arima(
    median_income_pivot_from_1999, 2019, n_periods=2, model_desc="From 1999"
)

Training From 1999 2019: 100%|██████████| 34/34 [00:26<00:00,  1.26it/s]


In [ ]:
metrics_df = pd.DataFrame(
    {
        "From 1999": calculate_metrics_partial(from_1999_pred_last_2),
        "lg From 1999": calculate_metrics_partial(lg_from_1999_pred_last_2.map(np.exp)),
    }
)
metrics_df

,From 1999,lg From 1999
mean_absolute_error,2.244628,2.212982
mean_absolute_percentage_error,0.036033,0.034276
median_absolute_error,1.512170,1.279168
mean_squared_error,8.864779,11.168192
mean_squared_log_error,0.002045,0.002313
r2_score,0.968242,0.959991


The log-transformed data from 1999-2018 yielded the lower error predictions for 2019 and 2020 median income across 34 sub-districts, with the lower mean absolute error (MAE) and mean absolute percentage errors (MAPE). The mean absolute percentage error is especially relevant as it reflects the relative accuracy of predictions.

We will now apply thessame strategy and use the `auto_arima` algorithm again to estimate the unknown median values, lower quartile values and upper quartile values for the years 2021 and 2022. In doing this all our datasets would have those roster years in common for easy alignment.

In [ ]:
def forecast_and_convert(df, start_year, periods, model_desc, column_name):
    """Forecast using autoarima and convert to long format."""
    lg_pred = hf.forecast_arima(
        df, start_year, n_periods=periods, model_desc=model_desc
    )
    long_format = convert_to_long_format(lg_pred.map(np.exp), column_name)
    return long_format


# Forecast and convert to long format
long_format_median_income_21_22 = forecast_and_convert(
    lg_median_income_pivot_from_1999, 2021, 2, "Log Model 1999", "median_income"
)
long_format_lower_q_income_21_22 = forecast_and_convert(
    lg_lower_q_income_pivot_from_1999,
    2021,
    2,
    "Log lower_q Model 1999",
    "lower_q_income",
)
long_format_upper_q_income_21_22 = forecast_and_convert(
    lg_upper_q_income_pivot_from_1999,
    2021,
    2,
    "Log upper_q Model 1999",
    "upper_q_income",
)

Training Log upper_q Model 1999 2021: 100%|██████████| 34/34 [00:33<00:00,  1.00it/s]


In [ ]:
# merge the three forecasted dataframes
income_forecast_df = long_format_median_income_21_22.merge(
    long_format_lower_q_income_21_22, on=["sub_district", "roster"]
).merge(long_format_upper_q_income_21_22, on=["sub_district", "roster"])

income_forecast_df.head()

,sub_district,roster,median_income,lower_q_income,upper_q_income
0,011,2021-01-01,60.700000,29.693798,92.976154
1,011,2022-01-01,60.700000,29.521613,94.085691
2,012,2021-01-01,86.100000,43.000000,161.750000
3,012,2022-01-01,86.100000,43.000000,161.750000
4,013,2021-01-01,84.553793,46.557861,144.960911


In [ ]:
# Concatenate the forecasted dataframes with the original dataframe
income_from_1999_with_forcasted = pd.concat(
    [income_from_1999, income_forecast_df], axis=0
)[["roster", "sub_district", "median_income", "lower_q_income", "upper_q_income"]]

# Align the roster years with the dog_data
income_from_1999_with_forcasted = hf.query_for_time_period(
    income_from_1999_with_forcasted
)

In [ ]:
# save the processed income data to data folder
income_from_1999_with_forcasted.to_csv("../data/processed_income_data.csv", index=False)

In [ ]:
(
    plot_arima_forecast(
        income_from_1999_with_forcasted,
        "median_income",
        vline=2020,
        non_forecast_color="blue",
    ).opts(height=800, active_tools=["box_zoom"])
    * plot_arima_forecast(
        income_from_1999_with_forcasted,
        "upper_q_income",
        vline=2020,
        non_forecast_color="green",
    )
).opts(height=800, active_tools=["box_zoom"])

:Overlay
   .NdOverlay.I   :NdOverlay   [sub_district]
      :Curve   [roster]   (median_income)
   .NdOverlay.II  :NdOverlay   [sub_district]
      :Curve   [roster]   (median_income)
   .VLine.I       :VLine   [x,y]
   .NdOverlay.III :NdOverlay   [sub_district]
      :Curve   [roster]   (upper_q_income)
   .NdOverlay.IV  :NdOverlay   [sub_district]
      :Curve   [roster]   (upper_q_income)
   .VLine.II      :VLine   [x,y]

#### Zurich Household Dataset

For the household datasets we first rename some of the columns so that they are more readable and consistent with the other datasets. We then process it to obtain an average household size per neighborhood, weighted by the number of households.

In [ ]:
# Define a dictionary to map old column names to new ones
column_rename_dict = {
    "quar_lang": "neighborhood",
    "key_day_dat_year": "roster",
    "number_hh": "household_count",
    "number_we": "resident_count",
}

# Rename the columns
zurich_household_data.rename(columns=column_rename_dict, inplace=True)

# Create new columns
zurich_household_data["sub_district"] = (
    zurich_household_data["quar_sort"].astype("string").str.zfill(3)
)
zurich_household_data["district"] = (
    zurich_household_data["sub_district"].str[:2].astype(int)
)
zurich_household_data["household_size"] = (
    zurich_household_data["hh_size_sort"].astype("string").str.zfill(2)
)

# Create a dataframe with only the columns of interest
columns_of_interest_household = [
    "neighborhood",
    "roster",
    "district",
    "sub_district",
    "household_size",
    "household_count",
    "resident_count",
]
hh_data = zurich_household_data[columns_of_interest_household]

# Align the roster years with the dog_data
hh_data = hf.query_for_time_period(hh_data)

# Display dataframe info and first 10 rows
hh_data.info()
hh_data.head(10)

<class 'pandas.core.frame.DataFrame'>
Index: 1632 entries, 408 to 2039
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   neighborhood     1632 non-null   object
 1   roster           1632 non-null   int64 
 2   district         1632 non-null   int32 
 3   sub_district     1632 non-null   string
 4   household_size   1632 non-null   string
 5   household_count  1632 non-null   int64 
 6   resident_count   1632 non-null   int64 
dtypes: int32(1), int64(3), object(1), string(2)
memory usage: 95.6+ KB


,neighborhood,roster,district,sub_district,household_size,household_count,resident_count
408,Rathaus,2015,1,011,01,1151,1151
409,Rathaus,2015,1,011,02,501,1002
410,Rathaus,2015,1,011,03,129,387
411,Rathaus,2015,1,011,04,79,316
412,Rathaus,2015,1,011,05,18,90
413,Rathaus,2015,1,011,06,11,69
414,Hochschulen,2015,1,012,01,109,109
415,Hochschulen,2015,1,012,02,69,138
416,Hochschulen,2015,1,012,03,14,42
417,Hochschulen,2015,1,012,04,17,68


In [ ]:
# Calculate total residents per sub_district
hh_data["total_residents"] = hh_data.groupby(["roster", "sub_district"])[
    "resident_count"
].transform("sum")

# Calculate the total households per sub_district
hh_data["total_households"] = hh_data.groupby(["roster", "sub_district"])[
    "household_count"
].transform("sum")

# Average household size
hh_data["avg_household_size"] = hh_data["total_residents"] / hh_data["total_households"]

# Calculate weighted average household size
hh_data["resident_portion"] = hh_data["resident_count"] / hh_data["total_residents"]
# household_data

hh_data = (
    hh_data.groupby(["roster", "sub_district", "neighborhood"])[
        ["avg_household_size", "total_households"]
    ]
    .mean()
    .reset_index()
)
hh_data.info()
hh_data.describe(include="all").T.sort_values(by="unique").infer_objects(
    copy=False
).fillna("")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 272 entries, 0 to 271
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   roster              272 non-null    int64  
 1   sub_district        272 non-null    string 
 2   neighborhood        272 non-null    object 
 3   avg_household_size  272 non-null    float64
 4   total_households    272 non-null    float64
dtypes: float64(2), int64(1), object(1), string(1)
memory usage: 10.8+ KB


,count,unique,top,freq,mean,std,min,25%,50%,75%,max
sub_district,272.0,34.0,011,8.0,,,,,,,
neighborhood,272.0,34.0,Rathaus,8.0,,,,,,,
roster,272.0,,,,2018.5,2.295511,2015.0,2016.75,2018.5,2020.25,2022.0
avg_household_size,272.0,,,,1.999885,0.208769,1.591141,1.88464,1.988107,2.062673,2.571385
total_households,272.0,,,,6020.794118,3948.082325,217.0,3041.75,5207.5,9008.75,17523.0


In [ ]:
# save the processed household data to data folder
hh_data.to_csv("../data/processed_household_data.csv", index=False)

#### Merged Datasets
Now that we have all datasets with some common columns we can attempt to merge them just to see if anything stands out. Since all of our processed files will save to the data folder we can simply just load them.

In [ ]:
# Load processed data from CSV files
neighborhood_gdf = gpd.read_file("../data/zurich_neighborhoods.geojson")
districts_gdf = gpd.read_file("../data/zurich_districts.geojson")
processed_dog_data = pd.read_csv("../data/processed_dog_data.csv")
processed_pop_data = pd.read_csv("../data/processed_pop_data.csv")
processed_income_data = pd.read_csv("../data/processed_income_data.csv")
processed_household_data = pd.read_csv("../data/processed_household_data.csv")

# Display the last 5 rows of the processed dog data
processed_dog_data.tail()

# Pad 'sub_district' column with leading zeros to make it 3 digits
processed_dog_data["sub_district"] = (
    processed_dog_data["sub_district"].astype("string").str.zfill(3)
)

# Group dog data by 'roster' and 'sub_district' and calculate the size of each group
grouped_dog_data = processed_dog_data.groupby(["roster", "sub_district"])
dog_count_to_merge = grouped_dog_data.size().rename("dog_count").reset_index()

# Count unique 'owner_id' in each group
owner_count_to_merge = (
    grouped_dog_data["owner_id"].nunique().rename("owner_count").reset_index()
)

# Count the number of small dogs (dog_size='K') in each group
small_dog_count_to_merge = (
    processed_dog_data.loc[processed_dog_data["dog_size"] == "K"]
    .groupby(["roster", "sub_district"])
    .size()
    .rename("small_dog_count")
    .reset_index()
)
# Count the number of pure breed dogs in each group
pure_breed_count_to_merge = (
    processed_dog_data.loc[processed_dog_data["is_pure_breed"]]
    .groupby(["roster", "sub_district"])
    .size()
    .rename("pure_breed_count")
    .reset_index()
)
# Count the number of male owners in each group
male_owner_count_to_merge = (
    processed_dog_data.loc[processed_dog_data["is_male_owner"] == True]
    .groupby(["roster", "sub_district"])["owner_id"]
    .nunique()
    .reset_index(name="male_owner_count")
)
# Pad 'sub_district' column in population data and group by 'roster' and 'sub_district'
processed_pop_data["sub_district"] = (
    processed_pop_data["sub_district"].astype("string").str.zfill(3)
)
pop_to_merge = (
    processed_pop_data.groupby(["roster", "sub_district"])
    .agg({"pop_count": "sum"})
    .reset_index()
)

# Pad 'sub_district' column in income data, truncate 'roster' to 4 digits, and group by 'roster' and 'sub_district'
processed_income_data["sub_district"] = (
    processed_income_data["sub_district"].astype("string").str.zfill(3)
)
processed_income_data["roster"] = processed_income_data["roster"].str[:4].astype(
    int)
income_to_merge = (
    processed_income_data.groupby(["roster", "sub_district"])
    .agg({"median_income": "mean", "lower_q_income": "mean", "upper_q_income": "mean"})
    .round(3)
    .reset_index()
)

# Pad 'sub_district' column in household data and group by 'roster' and 'sub_district'
processed_household_data["sub_district"] = (
    processed_household_data["sub_district"].astype("string").str.zfill(3)
)
hh_to_merge = (
    processed_household_data.groupby(["roster", "sub_district"])
    .agg({"avg_household_size": "mean", "total_households": "mean"})
    .round(3)
    .reset_index()
)

# Merge all the grouped data into a single DataFrame
z_subd_merged = (
    dog_count_to_merge.merge(owner_count_to_merge)
    .merge(male_owner_count_to_merge)
    .merge(small_dog_count_to_merge)
    .merge(pure_breed_count_to_merge)
    .merge(pop_to_merge)
    .merge(income_to_merge)
    .merge(hh_to_merge)
    .merge(neighborhood_gdf[["sub_district", "district", "subd_area_km2"]])
)

In [ ]:
z_subd_merged["small_dog_frac"] = round(
    z_subd_merged["small_dog_count"] / z_subd_merged["dog_count"], 3
)
# Add in the owner to population ratio
z_subd_merged["owner_pop_ratio"] = (
    z_subd_merged["owner_count"] / z_subd_merged["pop_count"]
)


# Add in the geometry data and subd_area_km2 for density calculations
z_subd_merged["dog_subd_density"] = (
    z_subd_merged["dog_count"] / z_subd_merged["subd_area_km2"]
)
z_subd_merged["hh_subd_density"] = (
    z_subd_merged["total_households"] / z_subd_merged["subd_area_km2"]
)
z_subd_merged["pop_subd_density"] = (
    z_subd_merged["pop_count"] / z_subd_merged["subd_area_km2"]
)
z_subd_merged["owner_subd_density"] = (
    z_subd_merged["owner_count"] / z_subd_merged["subd_area_km2"]
)

# z_merged

##### Dimensionality Reduction: UMAP vs PCA

When dealing with high-dimensional data, dimensionality reduction techniques like UMAP (Uniform Manifold Approximation and Projection) and PCA (Principal Component Analysis) are often used. These techniques transform the data into a lower-dimensional space, making it easier to visualize and analyze.
##### PCA (Principal Component Analysis)
**PCA** is a linear technique that focuses on preserving the global structure of the data, which refers to the overall variance in the data. It's computationally efficient but may not always capture the relationships in the data when reducing to very low dimensions.

On the other hand, **UMAP** is a nonlinear technique that aims to preserve both the global and local structure of the data. The local structure refers to the relationships in the data when reducing to very low dimensions, like 2D or 3D. This makes UMAP potentially more effective than PCA for visualization purposes, but it comes at the cost of higher computational intensity.

Here's a summary of the key differences:

| | UMAP | PCA |
|---|---|---|
| **Preserves Global Structure** | Yes | Yes |
| **Preserves Local Structure** | Yes | No |
| **Computational Intensity** | High | Low |

In [ ]:
# List the columns to be used on PCA analysis
columns_to_use = [
    "dog_subd_density",
    "hh_subd_density",
    "pop_subd_density",
    "owner_subd_density",
    "dog_count",
    "pop_count",
    "owner_count",
    "male_owner_count",
    "pure_breed_count",
    "owner_pop_ratio",
    "total_households",
    "median_income",
    "avg_household_size",
    "small_dog_frac",
    "small_dog_count",
    # "subd_area_km2",
    # "lower_q_income",
    # "upper_q_income",
]

# State the range for the number of clusters to be used for the KMeans algorithm
n_clusters = range(2, 13)

# scale the data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(z_subd_merged[columns_to_use])
scaled_data_df = pd.DataFrame(scaled_data, columns=columns_to_use)

# Fit the PCA model
pca = PCA()
pca.fit(scaled_data_df)


# Plot the cumulative explained variance ratio with number of components
n = pca.n_components_
cumulative_variance_ratio = np.r_[0, pca.explained_variance_ratio_.cumsum()]
cumulative_variance_ratio_plot = hv.Curve(
    cumulative_variance_ratio, label="Cumulative Explained Variance Ratio"
)
cumulative_variance_ratio_scatter = hv.Scatter(
    cumulative_variance_ratio_plot.data
).opts(size=10, color="red", alpha=0.5)
cumulative_variance_ratio_plot.opts(
    title="Cumulative Explained Variance Ratio",
    xlabel="Number of Components",
    ylim=(0, 1),
    xlim=(0, n + 1),
    height=600,
    width=600,
    tools=["hover"],
    show_legend=False,
)
cumulative_variance_ratio_plot * cumulative_variance_ratio_scatter

:Overlay
   .Curve.Cumulative_Explained_Variance_Ratio :Curve   [x]   (y)
   .Scatter.I                                 :Scatter   [x]   (y)

We then selected only the principal components which accounted for 95% of the cumulative explained variance ratio. Using these reduced number of  dimensions, we were then able to perform a `Kmeans` cluster analysis to identify clusters of our `sub_districts` with similar features. Although we have 34 neighborhoods within 12 districts they may not necessarily be clustered along those lines. To assess the quality of our clustering results, we use the `silhouette score`.

In [ ]:
# transform the data
pca_data = pca.transform(scaled_data_df)
pca_data_df = pd.DataFrame(
    pca_data, columns=[f"PC{i}" for i in range(1, n + 1)])
pca_data_df


# Get the number of components that explain at least 95% of the variance
num_components = np.where(cumulative_variance_ratio >= 0.95)[0][0]

# Select the first `num_components` columns
pca_data_df_reduced = pca_data_df.iloc[:, :num_components]
# pca_data_df_reduced
print(f"{pca_data_df_reduced.shape[1]} components explain 95% of the variance")
pca_data_df_reduced.head()

5 components explain 95% of the variance


,PC1,PC2,PC3,PC4,PC5
0,-3.023821,2.677006,1.217219,0.008289,0.169891
1,-4.474466,-3.363613,0.172471,-0.711588,3.412067
2,-3.989028,-0.329701,1.957507,1.770259,-0.469449
3,-3.831648,-2.638659,-1.367044,-0.265617,0.114502
4,1.161055,-1.544714,-0.444786,0.325007,0.490674


In [ ]:
def get_pca_plots(nclusters, pca_data):
    """Create PCA plots for the given number of clusters."""
    cluster_labels = hf.compute_kmeans_labels(pca_data, nclusters)
    clustered_data_df = hf.create_clustered_data_df(pca_data, cluster_labels)
    clustered_data_df = hf.add_columns(
        clustered_data_df, z_subd_merged, ["sub_district", "district"]
    )
    plot = hf.create_scatterplot_with_origin_cross(
        clustered_data_df,
        x="PC1",
        y="PC2",
        title=f"K-means Clustering with k={nclusters}",
    )
    return plot

In [ ]:
# get the scores for the PCA data
pca_scores = hf.calculate_clusters_scores(
    {"PCA": pca_data_df_reduced}, n_clusters)


# draw an outline around the highest score in the silhouette score and calinski but the lowest in the davies
# pca_scores.set_index().style.apply(
#     lambda x: ["border: 2px solid red" if v == x.max() else "" for v in x],
#     axis=1,
# )
# pca_scores.style.highlight_max(axis=0, color="lightgreen").highlight_min(
#     axis=0, color="lightcoral"
# )
def color_max(val, df, col):
    color = "lightpink" if val == df[col].max() else "gray"
    return f"color: {color}"


def color_min(val, df, col):
    color = "skyblue" if val == df[col].min() else "gray"
    return f"color: {color}"


styled_df = (
    pca_scores.style.map(
        color_max, subset=["silhouette"], df=pca_scores, col="silhouette"
    )
    .map(color_min, subset=["davies_bouldin"], df=pca_scores, col="davies_bouldin")
    .map(
        color_max, subset=["calinski_harabasz"], df=pca_scores, col="calinski_harabasz"
    )
)

styled_df
# pca_scores.to_dict(orient="tight")

,embedding_key,n_clusters,silhouette,calinski_harabasz,davies_bouldin
0,PCA,2,0.293000,123,1.252000
1,PCA,3,0.350000,152,1.104000
2,PCA,4,0.282000,129,1.247000
3,PCA,5,0.295000,117,1.312000
4,PCA,6,0.357000,138,1.079000
5,PCA,7,0.307000,120,1.171000
6,PCA,8,0.375000,137,0.964000
7,PCA,9,0.362000,136,0.947000
8,PCA,10,0.359000,134,0.911000
9,PCA,11,0.354000,135,0.951000


We are able to see how the neighborhoods are clustered in the scatterplot below (on an arbitrary plane off the first 2 principle components), which is colored by the `cluster` label below. We also included two other metric which although we did not use, were still useful to look at. 
- `Calinski-Harabasz`: ratio of the between-clusters and inter-clusters dispersion for all clusters. The higher the value, the better the clustering.
- `Davies-Bouldin`: Similarity between clusters Comparing the distance between the clusters and the size of the clusters themselves. A lower score here relates to a model with better separation between the clusters.

In [ ]:
# list the cluster_metrics to be considered
cluster_metrics = ["silhouette", "calinski_harabasz", "davies_bouldin"]
# get the best cluster for each metric and plot the PCA data
hv.Layout(
    [
        get_pca_plots(
            pca_scores.loc[pca_scores[metric].idxmax()]["n_clusters"],
            pca_data=pca_data_df_reduced,
        ).opts(
            title=f"K-means || {pca_scores.loc[pca_scores[metric].idxmax()]['n_clusters']} clusters|| {metric}={pca_scores.loc[pca_scores[metric].idxmax()][metric]:.3f}",
        )
        for metric in cluster_metrics
    ]
).cols(3)

:Layout
   .Overlay.I   :Overlay
      .NdOverlay.I :NdOverlay   [cluster]
         :Scatter   [PC1]   (PC2)
      .VLine.I     :VLine   [x,y]
      .HLine.I     :HLine   [x,y]
   .Overlay.II  :Overlay
      .NdOverlay.I :NdOverlay   [cluster]
         :Scatter   [PC1]   (PC2)
      .VLine.I     :VLine   [x,y]
      .HLine.I     :HLine   [x,y]
   .Overlay.III :Overlay
      .NdOverlay.I :NdOverlay   [cluster]
         :Scatter   [PC1]   (PC2)
      .VLine.I     :VLine   [x,y]
      .HLine.I     :HLine   [x,y]

In [ ]:
# Get the best number of clusters for silhouette score
best_n_clusters = pca_scores.loc[pca_scores["silhouette"].idxmax(
)]["n_clusters"]
print(f"Best number of clusters Silhouette: {best_n_clusters}")

# Apply k-means clustering to the PCA data
kmeans = KMeans(n_clusters=best_n_clusters, random_state=628)
kmeans.fit(pca_data_df_reduced)
pca_data_df_reduced["cluster"] = kmeans.labels_

# add columns district, sub_district, and roster
pca_data_df_reduced = hf.add_columns(
    pca_data_df_reduced, z_subd_merged, ["sub_district", "district", "roster"]
)


# Group the DataFrame by 'sub_district' and get the set of 'cluster' for each 'sub_district'
sub_district_pca_cluster = (
    pca_data_df_reduced[["sub_district", "district", "roster", "cluster"]]
    .groupby(["sub_district"])["cluster"]
    .apply(set)
    .reset_index(name="pca_cluster_set")
)


# Calculate the number of unique clusters for each 'sub_district' and store it in a new column
sub_district_pca_cluster["sub_district_cluster_count"] = sub_district_pca_cluster[
    "pca_cluster_set"
].apply(len)

# Check if there are any 'sub_district' with more than one unique cluster and sum them
(~sub_district_pca_cluster["sub_district_cluster_count"] == 1).sum()

# unravel the set in the pcs column
sub_district_pca_cluster["pca_cluster"] = sub_district_pca_cluster[
    "pca_cluster_set"
].apply(lambda x: list(x)[0])


# Plot the Clusters (colormap) and Districts(white line) and sub_districts (black (default) line)
neighborhood_gdf.merge(
    sub_district_pca_cluster[["sub_district", "pca_cluster"]]
).hvplot.polygons(
    aspect="equal",
    geo=True,
    tiles="EsriImagery",
    color="pca_cluster",
    # cmap="glasbey_dark",
    colormap=cc.glasbey_dark[: best_n_clusters + 1],
    hover_cols=["all"],
    title=f"PCA || {best_n_clusters} Clusters",
    xaxis="bare",
    yaxis="bare",
    colorbar=False,
    alpha=0.6,
) * gv.Polygons(
    districts_gdf
).opts(
    line_color="white", line_width=2, fill_alpha=0.02, height=800, width=800
)

Best number of clusters Silhouette: 8


:Overlay
   .WMTS.I      :WMTS   [Longitude,Latitude]
   .Polygons.I  :Polygons   [Longitude,Latitude]   (pca_cluster)
   .Polygons.II :Polygons   [Longitude,Latitude]   (district,d_area_km2)

We can also use these cluster labels to see how the neighborhoods are distributed on a map. We can see that the clusters are not necessarily along the district lines (white outline) but still have a strong geographical contiguity. This clustering was based only on the non geometry features, meaning excluding the `area` and geographical coordinates as features for example. These of course will not make much sense for pca as they have no variation within them from year to year as the other features do. As our features also included `roster` which was the year feature, some `sub_districts` were classified into different clusters for different years. This may be due to some non linearity in our system.

In [ ]:
# Compute correlations between original variables and principal components

corr = pd.DataFrame(
    pca.components_.T[:, :num_components],
    columns=pca_data_df.columns[:num_components],
)
corr["variable"] = columns_to_use
corr = corr.melt(id_vars="variable", var_name="PC", value_name="corr")
# plot using hvplot
corr_plot = corr.hvplot.bar(
    x="PC",
    y="corr",
    by="variable",
    width=1200,
    height=500,
    title="Correlation Between Original Variables and Principal Components",
    ylabel="",
    tools=["hover"],
).opts(active_tools=["box_zoom"])
corr_plot.opts(xrotation=90, xlabel="", gridstyle={"grid_line_color": "lightgray"})

:Bars   [PC,variable]   (corr)

The bar plot helps us to understand The principal components in terms of the original variables.
- a high positive value means the original variable on the principal component are strongly positively correlated
- a high negative value means that the original variable are strongly negatively correlated.

We began to see a pattern emerging here with the 'density' features being highly correlated with other 'density' features, likewise for the counts and the 'ratios' features. This is shown more clearly in the circle correlation plot.

In [ ]:
correlations = pd.DataFrame(pca.components_, columns=scaled_data_df.columns).T
correlations.columns = [f"PC{i}" for i in range(1, n + 1)]
labels_df = pd.DataFrame(
    {"x": correlations["PC1"], "y": correlations["PC2"],
        "label": correlations.index}
)
circle_correlation = correlations.hvplot.scatter(
    x="PC1",
    y="PC2",
    width=800,
    height=500,
    title="Correlation Between Principal Components",
    hover_cols=["index"],
    xlim=(-1, 1),
)

(
    circle_correlation
    # plot the dog_subd_density point as a red color
    * labels_df.loc[labels_df["label"] == "dog_subd_density"]
    .hvplot.points()
    .opts(color="red")
    * hv.VLine(0).opts(color="gray", line_dash="dotted")
    * hv.HLine(0).opts(color="gray", line_dash="dotted")
    # * hv.Labels(labels_df, ["x", "y"], "label").opts(
    #     yoffset=-0.05, xoffset=-0.05, text_alpha=0.6
    # )
)

:Overlay
   .Scatter.I :Scatter   [PC1]   (PC2,index)
   .Points.I  :Points   [x,y]
   .VLine.I   :VLine   [x,y]
   .HLine.I   :HLine   [x,y]

In [ ]:
print(labels_df)

                           x         y               label
dog_subd_density   -0.007109  0.424085    dog_subd_density
hh_subd_density     0.000883  0.472790     hh_subd_density
pop_subd_density    0.011600  0.467926    pop_subd_density
owner_subd_density -0.008868  0.424356  owner_subd_density
dog_count           0.372464 -0.061787           dog_count
pop_count           0.369370  0.057098           pop_count
owner_count         0.371767 -0.057937         owner_count
male_owner_count    0.371955 -0.019725    male_owner_count
pure_breed_count    0.369595 -0.071366    pure_breed_count
owner_pop_ratio    -0.042272 -0.218930     owner_pop_ratio
total_households    0.363517  0.084364    total_households
median_income      -0.164326 -0.181189       median_income
avg_household_size  0.074572 -0.184200  avg_household_size
small_dog_frac      0.051701  0.243108      small_dog_frac
small_dog_count     0.377149 -0.027631     small_dog_count


##### UMAP (Uniform Manifold Approximation and Projection)
We now do a similar analysis using UMAP. The `UMAP` class has a few more parameters to play with than `PCA` so it can be slightly more confusing but you gain more control over the process.

In [ ]:
# declare a panel widget for buttons
n_neighbors_slider = pnw.IntSlider(
    value=15, start=5, end=50, step=5, width=400, name="n_neighbors"
)
n_clusters_slider = pnw.IntSlider(
    value=5, start=2, end=12, step=1, width=400, name="n_clusters"
)
min_dist_button = pnw.RadioButtonGroup(options=[0.1, 0.2, 0.4, 0.7], value=0.2)
# List of values to try for n_neighbors and min_dist
n_neighbors_values = list(range(5, 51, 5))
min_dist_values = [0.1, 0.2, 0.4, 0.7]

# Get the embeddings dictionary which
embeddings_dict = hf.compute_embeddings(
    scaled_data_df, n_neighbors_values, min_dist_values
)

In [ ]:
# embeddings_dict[(10, 0.5)]

In [ ]:
@pn.cache(max_items=20)
@pn.depends(n_neighbors_slider.param.value, n_clusters_slider.param.value)
def get_umap_plot(neighbor, n_clusters):
    """Returns a HoloViews scatter plot of the UMAP embeddings."""
    plots = []
    embeddings_keys = [(neighbor, min_distance) for min_distance in min_dist_values]

    for embedding_key in embeddings_keys:
        # Retrieve the specific embeddings from the dictionary using the key
        embeddings = embeddings_dict[embedding_key]

        # Compute the cluster labels for the current embeddings
        cluster_labels = hf.compute_kmeans_labels(embeddings, n_clusters)

        # Create a DataFrame that combines the embeddings and their corresponding cluster labels
        embeddings_df = hf.create_clustered_data_df(embeddings, cluster_labels)
        embeddings_df = hf.add_columns(
            embeddings_df, z_subd_merged, ["sub_district", "district"]
        )

        # Generate a plot for the current embeddings and append it to the list of plots
        plot = hf.create_scatterplot_with_origin_cross(
            embeddings_df,
            title=f"UMAP || {n_clusters=} || {neighbor=} || min_dist={embedding_key[1]}",
        )
        plots.append(plot)

    return hv.Layout(plots).cols(2).opts(shared_axes=False)

In [ ]:
clusters_scores_df = hf.calculate_clusters_scores(
    embeddings_dict, n_clusters
).sort_values(by="silhouette", ascending=False)

# get the embeddings for the best silhouette score,
# the best calinski_harabasz score and
best_silhouette_embeddings = clusters_scores_df.loc[
    clusters_scores_df["silhouette"].idxmax()
]
best_calinski_harabasz_embeddings = clusters_scores_df.loc[
    clusters_scores_df["calinski_harabasz"].idxmax()
]
# the best davies_bouldin score. Here we look for the minimum value
best_davies_bouldin_embeddings = clusters_scores_df.loc[
    clusters_scores_df["davies_bouldin"].idxmin()
]
print(
    f"""
Best Silhouette Score:{best_silhouette_embeddings["silhouette"]:.3f}\n
Best Calinski Harabasz Score:{best_calinski_harabasz_embeddings["calinski_harabasz"]:.0f}\n
Best Davies Bouldin Score:{best_davies_bouldin_embeddings["davies_bouldin"]:.3f}
"""
)
# display the 3 embeddings
display(best_silhouette_embeddings)
display(best_calinski_harabasz_embeddings)
display(best_davies_bouldin_embeddings)


Best Silhouette Score:0.787

Best Calinski Harabasz Score:8130

Best Davies Bouldin Score:0.302



embedding_key        (15, 0.2)
n_clusters                  10
silhouette               0.787
calinski_harabasz         5718
davies_bouldin           0.306
Name: 107, dtype: object

embedding_key        (20, 0.1)
n_clusters                  12
silhouette               0.693
calinski_harabasz         8130
davies_bouldin           0.418
Name: 142, dtype: object

embedding_key        (15, 0.2)
n_clusters                  11
silhouette               0.786
calinski_harabasz         5654
davies_bouldin           0.302
Name: 108, dtype: object

We calculate all the embeddings beforehand and store them in a dictionary so that our calls using the widget do not have to recompute them each time.

In [ ]:
# Create panel for UMAP plot
umap_panel = pn.pane.HoloViews(get_umap_plot)
pn.Column(pn.Row(n_neighbors_slider, n_clusters_slider), umap_panel)

BokehModel(combine_events=True, render_bundle={'docs_json': {'6b6d9c8a-ae0d-4c18-9a35-3a290c3ec49d': {'version…

In [ ]:
# get the embedding from embedding dict for the best silhouette score
best_silhouette_embeddings_key = best_silhouette_embeddings["embedding_key"]
best_silhouette_n_clusters = best_silhouette_embeddings["n_clusters"]

# get the actual embeddings
embeddings_of_best_silhouette = embeddings_dict[best_silhouette_embeddings_key]
print(f"Best silhouette embedding key: {best_silhouette_embeddings_key}")
print(f"Best silhouette n_clusters: {best_silhouette_n_clusters}")

# get the cluster labels for the best silhouette score
best_silhouette_cluster_labels = hf.compute_kmeans_labels(
    embeddings_of_best_silhouette, best_silhouette_embeddings["n_clusters"]
)

# see which sub_districts are in which cluster
best_silhouette_embeddings_df = hf.create_clustered_data_df(
    embeddings_of_best_silhouette, best_silhouette_cluster_labels
)
# Add in the sub_district, district, and roster columns
best_silhouette_embeddings_df = hf.add_columns(
    best_silhouette_embeddings_df, z_subd_merged, [
        "sub_district", "district", "roster"]
)

# group by cluster to see which subdistricts-roster combinations are in which cluster
clusters_sub_districts = (
    best_silhouette_embeddings_df.groupby(
        ["cluster", "roster"])["sub_district"]
    .apply(list)
    .reset_index(name="sub_districts_cluster")
)
# Ensure that each sub_district is only in one cluster
# clusters_sub_districts

Best silhouette embedding key: (15, 0.2)
Best silhouette n_clusters: 10


In [ ]:
# Group the DataFrame by 'sub_district' and get the set of 'cluster' for each 'sub_district'
sub_district_umap_cluster = (
    best_silhouette_embeddings_df.groupby("sub_district")["cluster"]
    .apply(set)
    .reset_index(name="umap_cluster_set")
)

# Calculate the number of unique clusters for each 'sub_district' and store it in a new column
sub_district_umap_cluster["sub_district_cluster_count"] = sub_district_umap_cluster[
    "umap_cluster_set"
].apply(len)


# Check if there are any 'sub_district' with more than one unique cluster and sum them
(~sub_district_umap_cluster["sub_district_cluster_count"] == 1).sum()
# unravel the set in the umap_cluster_set column
sub_district_umap_cluster["umap_cluster"] = sub_district_umap_cluster[
    "umap_cluster_set"
].apply(lambda x: list(x)[0])

# Calculate the number of subdistricts in each cluster and store it in a new column
sub_district_umap_cluster["cluster_count"] = sub_district_umap_cluster.groupby(
    ["umap_cluster"]
)["sub_district"].transform("count")

neighborhood_gdf.merge(
    sub_district_umap_cluster[["sub_district", "umap_cluster"]]
).hvplot.polygons(
    aspect="equal",
    geo=True,
    tiles="EsriImagery",
    color="umap_cluster",
    colormap=cc.glasbey_dark[: best_silhouette_n_clusters + 1],
    colorbar=False,
    hover_cols=["all"],
    xaxis="bare",
    yaxis="bare",
    alpha=0.6,
    title=f"UMAP Clusters || {best_silhouette_n_clusters} clusters",
) * gv.Polygons(
    districts_gdf
).opts(
    line_color="white", line_width=2, fill_alpha=0.02, height=800, width=800
)

:Overlay
   .WMTS.I      :WMTS   [Longitude,Latitude]
   .Polygons.I  :Polygons   [Longitude,Latitude]   (umap_cluster)
   .Polygons.II :Polygons   [Longitude,Latitude]   (district,d_area_km2)

With **UMAP**, we can see that our `sub_districts` did not migrate to different clusters as the `roster` year changed. This is a good sign as the algorithm was able to pick up on this similarity without us explicitly mentioning it and without explicitly giving it a feature like `area` which would have made it more obvious. 

This was something that the **PCA** was not able to pick up on with out the `area` feature, but the chloropleth maps for both of these Dimension Reduction algorithms looks very similar. This gives some proof of predictability to our data, and we can look into with more details in the exploratory data analysis phase.

In [ ]:
sub_district_umap_cluster.sort_values(by="cluster_count", ascending=False)
# best_silhouette_embeddings_df.cluster.value_counts()

,sub_district,umap_cluster_set,sub_district_cluster_count,umap_cluster,cluster_count
11,042,{1},1,1,7
27,102,{1},1,1,7
29,115,{1},1,1,7
7,031,{1},1,1,7
15,061,{1},1,1,7
9,034,{1},1,1,7
12,044,{1},1,1,7
19,073,{5},1,5,5
16,063,{5},1,5,5
6,024,{5},1,5,5


In [ ]:
# Check which features correlate with the dog density
corr = pd.DataFrame(scaled_data, columns=columns_to_use).corr()
corr["dog_subd_density"].sort_values().hvplot.barh(
    width=800, height=500, title="Correlation with Dog Density"
).opts(xlabel="", active_tools=["box_zoom"])
corr_dog_density = (
    corr["dog_subd_density"]
    .sort_values()
    .hvplot.barh(width=500, height=500, title="Correlation with Dog Density")
    .opts(active_tools=["box_zoom"])
)


# Get a corr bar plot for the dog count for next to the dog density plot
corr_dog_count = (
    corr["dog_count"]
    .sort_values()
    .hvplot.barh(
        width=500, height=500, title="Correlation with Dog Count", xlim=(None, 1)
    )
    .opts(active_tools=["box_zoom"])
)
corr_dog_density + corr_dog_count

:Layout
   .Bars.Dog_subd_density :Bars   [index]   (dog_subd_density)
   .Bars.Dog_count        :Bars   [index]   (dog_count)

In [ ]:
print(
    corr[["dog_subd_density"]]
    .merge(corr["dog_count"], left_index=True, right_index=True)
    .round(3)
)
# print(corr.round(3))

                    dog_subd_density  dog_count
dog_subd_density               1.000     -0.058
hh_subd_density                0.847     -0.118
pop_subd_density               0.828     -0.097
owner_subd_density             0.997     -0.062
dog_count                     -0.058      1.000
pop_count                      0.017      0.901
owner_count                   -0.049      0.999
male_owner_count               0.003      0.976
pure_breed_count              -0.070      0.997
owner_pop_ratio               -0.046      0.082
total_households               0.064      0.882
median_income                 -0.115     -0.269
avg_household_size            -0.388      0.172
small_dog_frac                 0.266     -0.001
small_dog_count               -0.036      0.983


In [ ]:
merged_heatmap_corr = corr.hvplot.heatmap(
    rot=90,
    height=500,
    cmap="RdBu",
    symmetric=True,
    title="Correlation Heatmap of Merged Data",
).opts(color_levels=5, active_tools=["box_zoom"])

annotated_heatmap_corr = hv.Text("dog_subd_density", "dog_count", "+").opts(
    text_color="black", text_font_size="18pt"
)
merged_heatmap_corr * annotated_heatmap_corr

:Overlay
   .HeatMap.I :HeatMap   [columns,index]   (value)
   .Text.I    :Text   [x,y]

In [ ]:
corr.round(4).to_dict()

{'dog_subd_density': {'dog_subd_density': 1.0,
  'hh_subd_density': 0.8472,
  'pop_subd_density': 0.8281,
  'owner_subd_density': 0.997,
  'dog_count': -0.0584,
  'pop_count': 0.0167,
  'owner_count': -0.0494,
  'male_owner_count': 0.0034,
  'pure_breed_count': -0.0696,
  'owner_pop_ratio': -0.0457,
  'total_households': 0.0638,
  'median_income': -0.1152,
  'avg_household_size': -0.3877,
  'small_dog_frac': 0.2657,
  'small_dog_count': -0.0361},
 'hh_subd_density': {'dog_subd_density': 0.8472,
  'hh_subd_density': 1.0,
  'pop_subd_density': 0.9901,
  'owner_subd_density': 0.8525,
  'dog_count': -0.1183,
  'pop_count': 0.1304,
  'owner_count': -0.1077,
  'male_owner_count': -0.0383,
  'pure_breed_count': -0.1384,
  'owner_pop_ratio': -0.4667,
  'total_households': 0.1906,
  'median_income': -0.3215,
  'avg_household_size': -0.3735,
  'small_dog_frac': 0.3866,
  'small_dog_count': -0.0672},
 'pop_subd_density': {'dog_subd_density': 0.8281,
  'hh_subd_density': 0.9901,
  'pop_subd_densit

Oddly enough, we can see that not many other features besides the 'density' features have a high correlation with the 'dog density' features. Similarly, the `dog_count` Feature had a high correlation with the other count features. We can use this double advantage and we will look more into more features and feature engineering in the next notebook.

In [ ]:
sub_district_umap_cluster
# cluster_sub_d

,sub_district,umap_cluster_set,sub_district_cluster_count,umap_cluster,cluster_count
0,011,{3},1,3,4
1,012,{5},1,5,3
2,013,{7},1,7,1
3,014,{5},1,5,3
4,021,{2},1,2,3
5,023,{0},1,0,3
6,024,{6},1,6,7
7,031,{1},1,1,7
8,033,{0},1,0,3
9,034,{1},1,1,7


In [ ]:
z_subd_merged
sub_district_umap_cluster.merge(
    sub_district_pca_cluster,
).rename(
    columns={
        "sub_district_cluster_count": "pca_cluster_count",
        "cluster_count": "umap_cluster_count",
    }
)


cluster_sub_d = clusters_sub_districts[["cluster", "sub_districts_cluster"]]
cluster_sub_d["sub_districts_cluster"] = cluster_sub_d["sub_districts_cluster"].apply(
    frozenset
)
cluster_sub_d = cluster_sub_d.drop_duplicates()
cluster_sub_d["sub_districts_cluster"] = cluster_sub_d["sub_districts_cluster"].apply(
    set
)
sub_district_umap_cluster["cluster"] = sub_district_umap_cluster["umap_cluster"].copy()
# sub_district_umap_cluster_to_merge = sub_district_umap_cluster.merge(cluster_sub_d)
# sub_d_umap_c = sub_district_umap_cluster_to_merge[
#     ["sub_district", "sub_districts_cluster"]
# ]

# cluster_sub_d
a_copy_dataframe = sub_district_umap_cluster.merge(cluster_sub_d)
a_copy_dataframe

,sub_district,umap_cluster_set,sub_district_cluster_count,umap_cluster,cluster_count,cluster,sub_districts_cluster
0,011,{3},1,3,4,3,"{011, 041, 051, 082}"
1,012,{5},1,5,3,5,"{081, 012, 014}"
2,013,{7},1,7,1,7,{013}
3,014,{5},1,5,3,5,"{081, 012, 014}"
4,021,{2},1,2,3,2,"{101, 091, 021}"
5,023,{0},1,0,3,0,"{033, 023, 074}"
6,024,{6},1,6,7,6,"{024, 083, 071, 052, 072, 073, 063}"
7,031,{1},1,1,7,1,"{102, 034, 115, 042, 061, 031, 044}"
8,033,{0},1,0,3,0,"{033, 023, 074}"
9,034,{1},1,1,7,1,"{102, 034, 115, 042, 061, 031, 044}"


In [ ]:
a_copy_dataframe

In [ ]:
sub_district_umap_cluster

,sub_district,umap_cluster_set,sub_district_cluster_count,umap_cluster,cluster_count,sub_districts_cluster_x,sub_districts_cluster_y,sub_districts_cluster
0,011,{1},1,1,4,{},"{011, 041, 051, 082}","{011, 041, 051, 082}"
1,012,{9},1,9,2,{},"{012, 014}","{012, 014}"
2,013,{8},1,8,2,{},"{081, 013}","{081, 013}"
3,014,{9},1,9,2,{},"{012, 014}","{012, 014}"
4,021,{2},1,2,3,{},"{101, 091, 021}","{101, 091, 021}"
5,023,{4},1,4,2,{},"{033, 023}","{033, 023}"
6,024,{7},1,7,5,{},"{024, 083, 052, 073, 063}","{052, 063, 024, 083, 073}"
7,031,{3},1,3,7,{},"{102, 034, 115, 042, 061, 031, 044}","{034, 115, 042, 061, 044, 102, 031}"
8,033,{4},1,4,2,{},"{033, 023}","{033, 023}"
9,034,{3},1,3,7,{},"{102, 034, 115, 042, 061, 031, 044}","{034, 115, 042, 061, 044, 102, 031}"


In [ ]:
# Assuming clusters_grouped is your DataFrame and it has columns 'cluster' and 'sub_districts_cluster'

# Create an empty DataFrame
df = pd.DataFrame(columns=['sub_district', 'other_sub_districts'])

# Iterate over each row in the DataFrame
for _, row in clusters_grouped.iterrows():
    # Get the list of sub-districts in the same cluster
    sub_districts = row['sub_districts_cluster']

    # For each sub-district, create a DataFrame row where the 'sub_district' is the sub-district
    # and 'other_sub_districts' is a list of other sub-districts in the same cluster
    for sub_district in sub_districts:
        other_sub_districts = [
            sd for sd in sub_districts if sd != sub_district]
        df = df.append({'sub_district': sub_district,
                       'other_sub_districts': other_sub_districts}, ignore_index=True)

# Print the DataFrame
print(df)